<a href="https://colab.research.google.com/github/shintaroudlotulhanafia/DRLforMultipleStocksTradingUsingFinRL/blob/main/StockFractionRobustness/65_Fraksi4_TugasAkhirS1_Shinta_DRLforMultipleStocksTradingUsingFinRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data saham-saham fraksi 4 dijalankan menggunakan email shintarh18@gmail.com

# Deep Reinforcement Learning untuk Jual-Beli Saham (Dari Awal): Jual-beli Banyak Saham 

* **Pytorch Version** 



**Memasangkan Google Drive ke Google Colab**

* Kata kunci **from** digunakan untuk mengimpor hanya bagian tertentu dari modul (melakukan impor untuk bagian tertentu pada *library*)
* Modul merupakan file yang berisi sekumpulan fungsi yang ingin disertakan dalam aplikasi. Untuk membuat modul cukup simpan kode yang Anda inginkan dalam file dengan ekstensi file .py.
* **import** melakukan impor seluruh *library*.
* **Mounting** adalah proses membuat file dan direktori pada perangkat penyimpanan (seperti google drive) sehingga tersedia bagi pengguna untuk dapat diakses.



In [93]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Daftar Isi:

* [1. Pendeskrisian Tugas](#0)
* [2. Instalasi Paket Python](#1)
    * [2.1. Instalasi Paket](#1.1)    
    * [2.2. Daftar Paket Python](#1.2)
    * [2.3. Impot Paket](#1.3)
    * [2.4. Membuat Folder](#1.4)
* [3. Mengunduh Data](#2)
* [4. Memproses Data](#3)        
    * [4.1. IndiKator Teknikal](#3.1)
    * [4.2. Melakukan *Feature Engineering*](#3.2)
* [5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style](#4)  
    * [5.1. Pemisahan Data](#4.1)  
    * [5.2. Lingkungan untuk Pelatihan *(Training)*](#4.2)    
* [6. Melatih Agen DRL](#5)
    * [6.1. Agen 1: A2C](#5.1)
    * [6.2. Agen 2: PPO](#5.2)
    * [6.3. Agen 3: TD3](#5.3)
    * [6.4. Agen 4: SAC](#5.4)
    * [6.5. Agen 5: DDPG](#5.5)
* [7. Melakukan Jual-Beli](#6)
    * [7.1. Performa di Dalam Sampel](#6.1)
    * [7.2. Performa di Luar Sampel](#6.2)
    * [7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi](#6.3)
      * [7.3.1. Agen 1: A2C](#6.3.1)
      * [7.3.2. Agen 2: PPO](#6.3.2)
      * [7.3.3. Agen 3: TD3](#6.3.3)
      * [7.3.4. Agen 4: SAC](#6.3.4)
      * [7.3.5. Agen 5: DDPG](#6.3.5)
    * [7.4. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi (dalam bentuk string)](#6.3)
      * [7.4.1. Agen 1: A2C](#6.4.1)
      * [7.4.2. Agen 2: PPO](#6.4.2)
      * [7.4.3. Agen 3: TD3](#6.4.3)
      * [7.4.4. Agen 4: SAC](#6.4.4)
      * [7.4.5. Agen 5: DDPG](#6.4.5)
    * [7.5. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Kondisi dan Aksi](#6.4)
      * [7.5.1. Agen 1: A2C](#6.4.1)
      * [7.5.2. Agen 2: PPO](#6.4.2)
      * [7.5.3. Agen 3: TD3](#6.4.3)
      * [7.5.4. Agen 4: SAC](#6.4.4)
      * [7.5.5. Agen 5: DDPG](#6.4.5)
* [8. Performa Backtesting](#7)  
    * [8.1. Status BackTesting](#7.1)

<a id='0'></a>
# Bagian 1. Pendeskrisian Tugas

Agen DRL dilatih untuk melakukan jual-beli saham. Tugas tersebut dimodelkan sebagai Markov Decision Process (MDP), dengan fungsi dan tujuannya adalah memaksimalkan pengembalian pengembalian kumulatif *(cummulative return)* yang diharapkan.

Definisi state-action-reward pada algoritam DRL kasus kali ini adalah sebagai berikut:

* **State s**: Kondisi atau *state* mewakili persepsi agen tentang lingkungan pasar. Sama seperti *trader* manusia yang menganalisis berbagai informasi, agen juga secara pasif mengamati banyak fitur dan belajar dengan cara berinteraksi dengan lingkungan pasar (biasanya dengan menjalankan ulang data historis).

* **Tindakan atau *action* a**: Ruang aksi mencakup aksi atau tindakan yang dapat dilakukan agen di setiap status atau kondisi. Misalnya, a {−1, 0, 1}, -1 berarti menjual, 0 berarti menahan, dan 1 berarti membeli. Ketika suatu aksi mengoperasikan beberapa saham, maka, a {−k, ..., 1, 0, 1, ..., k}. Misalnya, "Beli
10 saham TLKM" atau "Jual 10 saham TLKM" maka masing-masing nilai a adalah 10 atau -10.

* **Fungsi *reward* atau imbalan r(s, a, s′)**: *Reward* adalah insentif bagi agen untuk mempelajari kebijakan yang lebih baik. Misalnya *reward* dapat berupa perubahan nilai portofolio saat mengambil a pada keadaan s dan tiba pada keadaan baru s', yaitu, r(s, a, s′) = v′-v, v′ mewakili nilai portofolio pada keadaan s′ dan v mewakili nilai portofolio pada keadaan s.

* **Environment atau lingkungan jual-beli**: saham penyusun indeks JII dengan tanggal sesuai periode pengujian yang diatur.


Data untuk studi kasus ini diperoleh dari Yahoo Finance API. Data berisi harga *Open-High-Low-Close* dan *Volume*.

<a id='1'></a>
# Bagian 2. Instalasi Paket Python

<a id='1.1'></a>
## 2.1. Instalasi Paket


* FinRL adalah *open-source framework* pertama yang menunjukkan potensi besar *Reinforcement Learning* keuangan.

* FinRL menyediakan berbagai pengaturan untuk melakukan *trading* dengan *Reinforcement Learning* seperti menyediakan ratusan pasar keuangan, algoritma yang canggih, berbagai macam aplikasi keuangan (alokasi portofolio, perdagangan mata uang kripto, *high-frequency trading*), *live trading, cloud deployment,* dll.

* Pengaplikasian pustaka FinRL dapat menggunakan alamat berikut, git+https://github.com/AI4Finance-Foundation/FinRL.git. Namun, untuk kemudahan pengaturan nilai sesuai kebutuhan, maka, dilakukanlah *forking* terhadap *repository* tersebut, lalu digunakanlah alamat berikut, git+https://github.com/shintaroudlotulhanafia/FinRL.git



In [94]:
# install finrl library
!pip install git+https://github.com/shintaroudlotulhanafia/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shintaroudlotulhanafia/FinRL.git to /tmp/pip-req-build-dvoq18m6
  Running command git clone -q https://github.com/shintaroudlotulhanafia/FinRL.git /tmp/pip-req-build-dvoq18m6
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-rk6nf7ju/pyfolio_93774938f82c4b739fd3847faf75370c
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-rk6nf7ju/pyfolio_93774938f82c4b739fd3847faf75370c
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-rk6nf7ju/elegantrl_02cb4e2df4c74a7db7078c49f59f26be
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-rk6nf7ju/elegantrl_02cb4e2df4c74a7db7078c49f59f26be


<a id='1.2'></a>
## 2.2. Daftar Paket Python
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Impot Paket

* Modul Python **pandas** digunakan untuk menganalisis dan memanipulasi data.
* Modul Python **numpy** digunakan untuk memproses larik atau array.
* Modul Python **matplotlib** digunakan membuat visualisasi data dalam dua dimensi.
* Modul Python **matplotlib.pyplot** adalah kumpulan fungsi yang membuat matplotlib berfungsi seperti MATLAB.
* Modul Python **Datetime** menyediakan sejumlah fungsi untuk menangani tanggal, waktu, dan interval waktu. Date dan datetime adalah objek dalam Python, bukan string atau timestamps.
* **YahooDownloader** menyediakan metode untuk mengambil data saham harian dari API Keuangan Yahoo!
* **FeatureEngineer** menyediakan metode untuk preprocessing data harga saham
* **data_split** membagi dataset menjadi data pelatihan dan data pengujian berdasarkan tanggal
* **StockTradingEnv** Lingkungan perdagangan saham untuk OpenAI gym
* **DRLAgent** menyediakan implementasi untuk algoritma DRL
* **DataProcessor** memproses data menggunakan prosesor data terpadu
* **backtest_stats** menghitung statistik *backtesting*
* **backtest_plot** membuat dan menampilkan plot ringkasan laporan *backtesting*.
* **get_daily_return** 
* **get_baseline** mengunduh data berdasarkan suatu indeks pada periode waktu tertentu.
* Metode **sys.path.append()** digunakan untuk menambahkan jalur sementara. Dengan demikian, jalur tersebut akan valid untuk sebuah sesi, misalnya.
* Python **itertool** adalah modul yang menyediakan berbagai fungsi yang bekerja pada iterator untuk menghasilkan iterator yang kompleks. Modul ini berfungsi sebagai alat yang cepat dan hemat memori.

In [95]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

<a id='1.4'></a>
## 2.4. Membuat Folder

* **config** berisi pengaturan tanggal periode training dan trading, indikator, hyperparameter setiap model/agen DRL.
* **config_tickers** berisi pengaturan daftar saham yang akan diproses.
* **import os** digunakan untuk meng-import modul, merupakan module pada python agar python itu sendiri berinteraksi langsung terhadap sistem operasi.
* **check_and_make_directories** digunakan untuk mengecek dan membuat folder.


In [96]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

<a id='2'></a>
# Part 3. Mengunduh Data

Yahoo Finance menyediakan data saham, berita keuangan, laporan keuangan, dll, secara gratis.
* FinRL *Library* menggunakan kelas **YahooDownloader** di FinRL-Meta untuk mengambil data melalui Yahoo Finance API
* Batas Pemanggilan: Menggunakan API Publik (tanpa autentikasi), Pengguna dibatasi hingga 2.000 permintaan per jam per IP (atau hingga total 48.000 permintaan per hari).

-----
kelas YahooDownloader:
    Mengambil data saham harian dari
    Yahoo Finance API

    Atribut
    ----------
        start_date : str
            tanggal mulai data
        end_date : str
            tanggal akhir data
        ticker_list : list
            daftar ticker saham (dimodifikasi dari config.py)

    Methods
    -------
    fetch_data()


Penetapan tanggal training dan trading dapat dilakukan dengan mengaturnya pada finrl/config.py atau dengan menetapkannya di sel notebook.

In [97]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2017-12-31'
TRADE_START_DATE = '2018-01-01'
TRADE_END_DATE = '2019-12-20'

In [98]:
#Data diperoleh dari idx.co.id diakses pada tanggal 27/08/2022
#Fraksi 1: <Rp200,00
fraksi1 = ['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

#Fraksi 2: Rp200,00 - Rp500,00
fraksi2 = ['BMTR.JK','BTON.JK','FORU.JK','GEMA.JK']

#Fraksi 3: Rp500,00 - Rp2.000,00
fraksi3 = ['AKRA.JK','BRPT.JK','KLBF.JK','MEDC.JK']

#Fraksi 4: Rp2000,00 - Rp5000,00
fraksi4 = ['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

#Fraksi 5: >Rp5000,00
fraksi5 =['INCO.JK','INDF.JK','INTP.JK','UNTR.JK']



In [99]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = fraksi4).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (10916, 8)


* **df.shape** digunakan untuk mendapatkan jumlah baris dan kolom

In [100]:
df.shape

(10916, 8)

* **nunique()** mengembalikan jumlah nilai unik untuk setiap kolom. Dengan menentukan sumbu kolom ( axis='columns' ), metode nunique() mencari berdasarkan kolom tersebut dan mengembalikan jumlah nilai unik untuk setiap baris.

In [101]:
df.nunique()

date      2729
open      2064
high      2070
low       2104
close     3389
volume    5390
tic          4
day          5
dtype: int64

* **unique()** digunakan untuk menemukan elemen unik dari array. Mengembalikan elemen unik yang diurutkan dari sebuah array.

In [102]:
print(df['tic'].unique())

['JECC.JK' 'TMAS.JK' 'TPIA.JK' 'UNVR.JK']


* **describe()** digunakan untuk menampilkan detail statistik dasar seperti persentil, mean, std, dll. Hanya kolom yang bertipe numerik yang akan ditampilkan statistiknya.

In [103]:
df.describe()

,open,high,low,close,volume,day
count,10916.000000,10916.000000,10916.000000,10916.000000,1.091600e+04,10916.000000
mean,2432.774045,2456.235431,2409.271419,2088.913528,5.551627e+06,1.988641
std,2995.412968,3020.367320,2969.115061,2648.204363,1.317312e+07,1.409951
min,30.000000,30.200001,27.000000,22.313637,0.000000e+00,0.000000
25%,152.000000,154.000000,149.000000,129.375214,1.000000e+03,1.000000
50%,875.503113,903.725342,855.000000,771.150024,3.977500e+05,2.000000
75%,3790.000000,3812.500000,3760.000000,3006.315674,7.077875e+06,3.000000
max,11235.000000,11620.000000,11180.000000,9885.863281,5.311050e+08,4.000000


* **sort_values()** berfungsi mengurutkan bingkai data dalam urutan *ascending* atau *descending* dari kolom yang dilewati.
* **ignore_index** berfungsi untuk menentukan untuk mereset indeks mulai dari nol. Secara default disetel *false*.
* **head()** digunakan untuk mendapatkan n baris pertama.

In [104]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-05,295.000000,295.000000,295.000000,198.915329,3,JECC.JK,0
1,2009-01-05,46.000000,47.000000,44.000000,34.214245,775000,TMAS.JK,0
2,2009-01-05,82.948853,82.948853,82.948853,66.105301,0,TPIA.JK,0
3,2009-01-05,1620.000000,1620.000000,1560.000000,1149.686401,20,UNVR.JK,0
4,2009-01-06,295.000000,295.000000,295.000000,198.915329,0,JECC.JK,1


<a id='3'></a>
# Bagian 4: Memproses Data

Periksa data yang hilang dan melakukan *feature engineering* untuk mengubah data menjadi *state*.
* **Menambahkan indikator teknis**

  Dalam praktik jual-beli, berbagai informasi perlu diperhitungkan, seperti harga historis, kepemilikan saham saat ini, indikator teknis, dll. Indikator teknis yang dapat digunakan antara lain, MACD, RSI, CCI, ADX, Bollinger Bands, dll. 
* **Menambahkan indeks turbulensi**

  Risk-aversion mencerminkan seberapa berani investor melakukan jual-beli saham dengan risiko yang tinggi. Hal ini mempengaruhi strategi trading investor ketika menghadapi tingkat volatilitas pasar yang berbeda. Untuk mengendalikan risiko dalam skenario terburuk, seperti krisis keuangan tahun 1998, FinRL menggunakan indeks turbulensi yang mengukur fluktuasi harga aset yang ekstrem.



-----
kelas FeatureEngineer:
    Menyediakan metode untuk preprocessing data harga saham

    Attributes
    ----------
        use_technical_indicator : boolean
            menggunakan indikator teknis atau tidak
        tech_indicator_list : list
            daftar nama indikator teknis (dimodifikasi dari neofinrl_config.py)
        use_vix : boolean
            menggunakan Volatility Index (VIX) atau tidak
        use_turbulence : boolean
            menggunakan indeks turbulensi atau tidak
        user_defined_feature:boolean
            menggunakan fitur yang ditentukan pengguna atau tidak
    Methods
    -------
    preprocess_data()
        metode utama untuk melakukan feature engineering


In [105]:
INDICATORS = [
    "macd",
    "rsi_30",
    "cci_30"
]

Membersihkan data mentah dari nilai-nilai yang hilang. Sehingga saham dengan data yang hilang dihapus dari daftar.

In [106]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2759, 8)
Successfully added vix
Successfully added turbulence index


In [107]:
processed.describe()

,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
count,10608.000000,10608.000000,10608.000000,10608.000000,1.060800e+04,10608.000000,10608.000000,10608.000000,10608.000000,10608.000000,10608.000000
mean,2429.045974,2452.658290,2405.377227,2085.151339,5.595889e+06,2.019231,9.631031,52.085155,6.622157,18.179936,4.051032
std,2992.801240,3017.942504,2966.282618,2645.361219,1.327819e+07,1.397923,61.035823,13.003422,126.156795,7.335005,11.274539
min,30.000000,30.200001,27.000000,22.313637,0.000000e+00,0.000000,-469.537055,0.000000,-1000.000000,9.140000,0.000000
25%,152.000000,154.000000,149.000000,129.375214,1.000000e+03,1.000000,-3.242396,45.273703,-71.650500,13.287500,0.301363
50%,860.000000,890.000000,830.000000,688.193024,4.074800e+05,2.000000,0.223007,51.351756,1.754121,15.985000,1.298436
75%,3762.500000,3790.000000,3730.000000,2984.897278,7.149700e+06,3.000000,15.008081,57.991457,81.626886,20.629999,3.926836
max,11235.000000,11620.000000,11005.000000,9885.863281,5.311050e+08,4.000000,507.194433,100.000000,1000.000000,56.650002,306.792718


In [108]:
processed.nunique()

date           2652
open           2044
high           2054
low            2085
close          3335
volume         5270
tic               4
day               5
macd          10592
rsi_30         7262
cci_30        10163
vix            1441
turbulence     2398
dtype: int64

* **tolist()** digunakan untuk mengubah elemen data array menjadi *list*.
* **unique()** digunakan untuk menemukan elemen unik dari array. Mengembalikan elemen unik yang diurutkan dari sebuah array.

In [109]:
list_ticker = processed["tic"].unique().tolist()
list_ticker

['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

* **date_range()** digunakan untuk mendapatkan frekuensi tetap DatetimeIndex. date_range() digunakan untuk membuat rentang tanggal di pandas.
* **max()** mengembalikan item dengan nilai tertinggi, atau item dengan nilai tertinggi dalam *iterable*. Jika nilainya adalah string, maka dilakukan perbandingan abjad.
* **min()** mengembalikan item dengan nilai terendah, atau item dengan nilai terendah dalam iterable. Jika nilainya adalah string,  maka dilakukan perbandingan abjad.
* **astype()** digunakan untuk mengubah tipe data dari suatu bentuk *series*.

In [110]:
list_date = list(pd.date_range(df['date'].min(),df['date'].max()).astype(str))
list_date

['2009-01-05',
 '2009-01-06',
 '2009-01-07',
 '2009-01-08',
 '2009-01-09',
 '2009-01-10',
 '2009-01-11',
 '2009-01-12',
 '2009-01-13',
 '2009-01-14',
 '2009-01-15',
 '2009-01-16',
 '2009-01-17',
 '2009-01-18',
 '2009-01-19',
 '2009-01-20',
 '2009-01-21',
 '2009-01-22',
 '2009-01-23',
 '2009-01-24',
 '2009-01-25',
 '2009-01-26',
 '2009-01-27',
 '2009-01-28',
 '2009-01-29',
 '2009-01-30',
 '2009-01-31',
 '2009-02-01',
 '2009-02-02',
 '2009-02-03',
 '2009-02-04',
 '2009-02-05',
 '2009-02-06',
 '2009-02-07',
 '2009-02-08',
 '2009-02-09',
 '2009-02-10',
 '2009-02-11',
 '2009-02-12',
 '2009-02-13',
 '2009-02-14',
 '2009-02-15',
 '2009-02-16',
 '2009-02-17',
 '2009-02-18',
 '2009-02-19',
 '2009-02-20',
 '2009-02-21',
 '2009-02-22',
 '2009-02-23',
 '2009-02-24',
 '2009-02-25',
 '2009-02-26',
 '2009-02-27',
 '2009-02-28',
 '2009-03-01',
 '2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-07',
 '2009-03-08',
 '2009-03-09',
 '2009-03-10',
 '2009-03-11',
 '2009-03-

* **Array** dapat menyimpan elemen hanya dari satu tipe. Sedangkan **list** juga dapat menyimpan elemen dari tipe data yang berbeda.
* **list()** digunakan untuk membuat objek *list*. Objek *list* adalah kumpulan yang berurutan dan dapat diubah.
* **itertools.product()** digunakan untuk mencari produk kartesius dari iterator yang diberikan, outputnya adalah urutan leksikografis.

In [111]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2009-01-05', 'JECC.JK'),
 ('2009-01-05', 'TMAS.JK'),
 ('2009-01-05', 'TPIA.JK'),
 ('2009-01-05', 'UNVR.JK'),
 ('2009-01-06', 'JECC.JK'),
 ('2009-01-06', 'TMAS.JK'),
 ('2009-01-06', 'TPIA.JK'),
 ('2009-01-06', 'UNVR.JK'),
 ('2009-01-07', 'JECC.JK'),
 ('2009-01-07', 'TMAS.JK'),
 ('2009-01-07', 'TPIA.JK'),
 ('2009-01-07', 'UNVR.JK'),
 ('2009-01-08', 'JECC.JK'),
 ('2009-01-08', 'TMAS.JK'),
 ('2009-01-08', 'TPIA.JK'),
 ('2009-01-08', 'UNVR.JK'),
 ('2009-01-09', 'JECC.JK'),
 ('2009-01-09', 'TMAS.JK'),
 ('2009-01-09', 'TPIA.JK'),
 ('2009-01-09', 'UNVR.JK'),
 ('2009-01-10', 'JECC.JK'),
 ('2009-01-10', 'TMAS.JK'),
 ('2009-01-10', 'TPIA.JK'),
 ('2009-01-10', 'UNVR.JK'),
 ('2009-01-11', 'JECC.JK'),
 ('2009-01-11', 'TMAS.JK'),
 ('2009-01-11', 'TPIA.JK'),
 ('2009-01-11', 'UNVR.JK'),
 ('2009-01-12', 'JECC.JK'),
 ('2009-01-12', 'TMAS.JK'),
 ('2009-01-12', 'TPIA.JK'),
 ('2009-01-12', 'UNVR.JK'),
 ('2009-01-13', 'JECC.JK'),
 ('2009-01-13', 'TMAS.JK'),
 ('2009-01-13', 'TPIA.JK'),
 ('2009-01-13', 'UNV

* **Pandas DataFrame** adalah dua dimensi yang dapat berubah ukuran, struktur data tabular yang berpotensi heterogen dengan sumbu berlabel (baris dan kolom).
* **merge()** berfungsi untuk memperbarui konten dua DataFrame dengan menggabungkannya bersama-sama, menggunakan metode yang ditentukan.

In [112]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915329,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.00000
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.00000
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105301,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.00000
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.00000
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915329,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15999,2019-12-18,UNVR.JK,8195.000000,8205.000000,8120.000000,7602.727051,15872500.0,2.0,-74.938448,41.772079,-169.670736,12.580000,0.11217
16000,2019-12-19,JECC.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16001,2019-12-19,TMAS.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16002,2019-12-19,TPIA.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* **isin()** digunakan untuk memfilter data frames. isin() dapat membantu untuk memilih baris dengan memiliki nilai tertentu (atau beberapa nilai tertentu) dalam kolom tertentu.

In [113]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915329,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105301,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915329,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNVR.JK,8180.000000,8225.000000,8180.000000,7612.015625,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.290000,0.019867
15996,2019-12-18,JECC.JK,6175.000000,6175.000000,6175.000000,6175.000000,0.0,2.0,-2.741289,53.058014,-164.670408,12.580000,0.112170
15997,2019-12-18,TMAS.JK,98.000000,100.000000,98.000000,80.050911,2135700.0,2.0,-3.212769,38.307115,-74.912976,12.580000,0.112170
15998,2019-12-18,TPIA.JK,2487.500000,2500.000000,2481.250000,2471.146240,22134800.0,2.0,45.007688,64.526583,113.802443,12.580000,0.112170


In [114]:
processed_full = processed_full.sort_values(['date','tic'])
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915329,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105301,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915329,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNVR.JK,8180.000000,8225.000000,8180.000000,7612.015625,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.290000,0.019867
15996,2019-12-18,JECC.JK,6175.000000,6175.000000,6175.000000,6175.000000,0.0,2.0,-2.741289,53.058014,-164.670408,12.580000,0.112170
15997,2019-12-18,TMAS.JK,98.000000,100.000000,98.000000,80.050911,2135700.0,2.0,-3.212769,38.307115,-74.912976,12.580000,0.112170
15998,2019-12-18,TPIA.JK,2487.500000,2500.000000,2481.250000,2471.146240,22134800.0,2.0,45.007688,64.526583,113.802443,12.580000,0.112170


* **fillna()** digunakan untuk mengisi nilai NA/NaN menggunakan metode yang ditentukan. fillna(0) artinya baris NA/NaN diisi dengan nilai misalnya 0.

In [115]:
processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915329,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105301,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915329,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNVR.JK,8180.000000,8225.000000,8180.000000,7612.015625,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.290000,0.019867
15996,2019-12-18,JECC.JK,6175.000000,6175.000000,6175.000000,6175.000000,0.0,2.0,-2.741289,53.058014,-164.670408,12.580000,0.112170
15997,2019-12-18,TMAS.JK,98.000000,100.000000,98.000000,80.050911,2135700.0,2.0,-3.212769,38.307115,-74.912976,12.580000,0.112170
15998,2019-12-18,TPIA.JK,2487.500000,2500.000000,2481.250000,2471.146240,22134800.0,2.0,45.007688,64.526583,113.802443,12.580000,0.112170


In [116]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915329,3.0,0.0,0.000000,0.0,-66.666667,39.080002,0.0
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.0,-66.666667,39.080002,0.0
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105301,0.0,0.0,0.000000,0.0,-66.666667,39.080002,0.0
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.0,-66.666667,39.080002,0.0
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915329,0.0,1.0,0.000000,0.0,-66.666667,38.560001,0.0
5,2009-01-06,TMAS.JK,46.000000,47.000000,44.000000,33.470455,1107500.0,1.0,-0.016688,0.0,-66.666667,38.560001,0.0
6,2009-01-06,TPIA.JK,82.948853,82.948853,82.948853,66.105301,0.0,1.0,0.000000,0.0,-66.666667,38.560001,0.0
7,2009-01-06,UNVR.JK,1620.000000,1670.000000,1600.000000,1163.880127,16432500.0,1.0,0.318449,100.0,66.666667,38.560001,0.0
8,2009-01-07,JECC.JK,270.000000,270.000000,270.000000,182.058121,10000.0,2.0,-0.522741,0.0,-100.000000,43.389999,0.0
9,2009-01-07,TMAS.JK,44.000000,48.000000,44.000000,32.726665,1765000.0,2.0,-0.044328,0.0,-18.789576,43.389999,0.0


In [117]:
processed_full.nunique()

date           2652
tic               4
open           2044
high           2054
low            2085
close          3335
volume         5270
day               5
macd          10592
rsi_30         7262
cci_30        10163
vix            1441
turbulence     2398
dtype: int64

* **info()** berfungsi untuk mencetak informasi tentang DataFrame. Informasi tersebut berisi jumlah kolom, label kolom, tipe data kolom, penggunaan memori, indeks rentang, dan jumlah sel di setiap kolom (nilai bukan nol).

In [118]:
processed_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10608 entries, 0 to 15999
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        10608 non-null  object 
 1   tic         10608 non-null  object 
 2   open        10608 non-null  float64
 3   high        10608 non-null  float64
 4   low         10608 non-null  float64
 5   close       10608 non-null  float64
 6   volume      10608 non-null  float64
 7   day         10608 non-null  float64
 8   macd        10608 non-null  float64
 9   rsi_30      10608 non-null  float64
 10  cci_30      10608 non-null  float64
 11  vix         10608 non-null  float64
 12  turbulence  10608 non-null  float64
dtypes: float64(11), object(2)
memory usage: 1.1+ MB


* **to_csv()** mengonversi DataFrame menjadi data CSV/comma separated value (nilai yang dipisahkan koma). Kita dapat melewatkan objek file untuk menulis data CSV ke dalam file. Jika tidak, data CSV dikembalikan dalam format string.
* **loc** berbasis label, yang berarti baris dan kolom diambil berdasarkan label baris dan kolomnya. **iloc** berbasis posisi integer, maka baris dan kolom dapat diambil dengan menggunakan nilai posisi integernya (posisi integer berbasis 0).

In [119]:
#Save the dataset by the tic filter
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  df_temp.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/DatasetsPerTic/Fraksi4/'+ stock +'.csv',index=False)
  display(df_temp)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.0,295.0,295.0,198.915329,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,295.0,295.0,295.0,198.915329,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
8,2009-01-07,JECC.JK,270.0,270.0,270.0,182.058121,10000.0,2.0,-0.522741,0.000000,-100.000000,43.389999,0.000000
12,2009-01-08,JECC.JK,250.0,250.0,250.0,168.572342,37500.0,3.0,-1.234582,0.000000,-104.761911,42.560001,0.000000
16,2009-01-09,JECC.JK,250.0,250.0,250.0,168.572342,0.0,4.0,-1.595394,0.000000,-79.710149,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,3.0,-3.619868,53.058014,-40.969081,13.940000,3.749015
15976,2019-12-13,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,4.0,-3.380631,53.058014,-139.322678,12.630000,0.593755
15988,2019-12-16,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,0.0,-3.154669,53.058014,-151.408554,12.140000,0.376882
15992,2019-12-17,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,1.0,-2.941682,53.058014,-167.378990,12.290000,0.019867


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
1,2009-01-05,TMAS.JK,46.0,47.0,44.0,34.214245,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
5,2009-01-06,TMAS.JK,46.0,47.0,44.0,33.470455,1107500.0,1.0,-0.016688,0.000000,-66.666667,38.560001,0.000000
9,2009-01-07,TMAS.JK,44.0,48.0,44.0,32.726665,1765000.0,2.0,-0.044328,0.000000,-18.789576,43.389999,0.000000
13,2009-01-08,TMAS.JK,42.0,45.0,42.0,31.239094,330000.0,3.0,-0.109521,0.000000,-133.333333,42.560001,0.000000
17,2009-01-09,TMAS.JK,44.0,44.0,42.0,32.726665,1590000.0,4.0,-0.083447,34.655537,-77.976644,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,TMAS.JK,99.0,100.0,97.0,78.433716,3130800.0,3.0,-3.753758,35.769412,-110.875793,13.940000,3.749015
15977,2019-12-13,TMAS.JK,98.0,100.0,98.0,80.050911,1143700.0,4.0,-3.607023,37.844101,-93.126705,12.630000,0.593755
15989,2019-12-16,TMAS.JK,100.0,100.0,98.0,79.242325,1726300.0,0.0,-3.515456,37.222229,-89.104401,12.140000,0.376882
15993,2019-12-17,TMAS.JK,99.0,99.0,97.0,79.242325,2965500.0,1.0,-3.403654,37.222229,-89.887110,12.290000,0.019867


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105301,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
6,2009-01-06,TPIA.JK,82.948853,82.948853,82.948853,66.105301,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
10,2009-01-07,TPIA.JK,82.948853,82.948853,82.948853,66.105301,61.0,2.0,0.000000,0.000000,-18.789576,43.389999,0.000000
14,2009-01-08,TPIA.JK,82.948853,82.948853,82.948853,66.105301,0.0,3.0,0.000000,0.000000,-133.333333,42.560001,0.000000
18,2009-01-09,TPIA.JK,82.948853,82.948853,82.948853,66.105301,0.0,4.0,0.000000,34.655537,-77.976644,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,TPIA.JK,2462.500000,2487.500000,2450.000000,2440.179688,32832800.0,3.0,37.659417,62.950307,136.531575,13.940000,3.749015
15978,2019-12-13,TPIA.JK,2468.750000,2487.500000,2462.500000,2452.566406,25854400.0,4.0,40.175702,63.571498,132.220628,12.630000,0.593755
15990,2019-12-16,TPIA.JK,2475.000000,2493.750000,2462.500000,2458.759521,20370000.0,0.0,42.183346,63.884688,123.628220,12.140000,0.376882
15994,2019-12-17,TPIA.JK,2481.250000,2506.250000,2475.000000,2464.952881,20856400.0,1.0,43.769622,64.203073,122.159603,12.290000,0.019867


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
3,2009-01-05,UNVR.JK,1620.0,1620.0,1560.0,1149.686401,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
7,2009-01-06,UNVR.JK,1620.0,1670.0,1600.0,1163.880127,16432500.0,1.0,0.318449,100.000000,66.666667,38.560001,0.000000
11,2009-01-07,UNVR.JK,1560.0,1660.0,1550.0,1107.105469,20.0,2.0,-1.354812,19.463155,-58.540316,43.389999,0.000000
15,2009-01-08,UNVR.JK,1590.0,1610.0,1570.0,1128.395874,20.0,3.0,-1.313958,38.635138,-59.838627,42.560001,0.000000
19,2009-01-09,UNVR.JK,1600.0,1630.0,1580.0,1135.493042,20.0,4.0,-0.981684,43.290492,-2.719662,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNVR.JK,8350.0,8420.0,8200.0,7616.660156,13365500.0,3.0,-49.981819,41.652679,-139.322678,13.940000,3.749015
15979,2019-12-13,UNVR.JK,8280.0,8300.0,8200.0,7658.458984,12891500.0,4.0,-56.350273,42.737208,-151.408554,12.630000,0.593755
15991,2019-12-16,UNVR.JK,8250.0,8275.0,8180.0,7598.082520,12503000.0,0.0,-65.513991,41.582279,-167.378990,12.140000,0.376882
15995,2019-12-17,UNVR.JK,8180.0,8225.0,8180.0,7612.015625,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.290000,0.019867


In [120]:
#memperbarui dataframe dengan bentuk harga per lot

processed_full_lot = processed_full[['open','high','low','close']].mul(100)

#drop some columns
processed_full_noOHLC = processed_full.drop(['open','high','low','close'], axis=1)
  
processed_full_lot = pd.concat([processed_full_noOHLC, processed_full_lot], axis=1)

processed_full_lot = processed_full_lot[['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'day', 'macd', 'rsi_30', 'cci_30', 'vix', 'turbulence']]

processed_full_lot

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,29500.000000,29500.000000,29500.000000,19891.532898,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,TMAS.JK,4600.000000,4700.000000,4400.000000,3421.424484,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,TPIA.JK,8294.885254,8294.885254,8294.885254,6610.530090,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,UNVR.JK,162000.000000,162000.000000,156000.000000,114968.640137,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,29500.000000,29500.000000,29500.000000,19891.532898,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNVR.JK,818000.000000,822500.000000,818000.000000,761201.562500,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.290000,0.019867
15996,2019-12-18,JECC.JK,617500.000000,617500.000000,617500.000000,617500.000000,0.0,2.0,-2.741289,53.058014,-164.670408,12.580000,0.112170
15997,2019-12-18,TMAS.JK,9800.000000,10000.000000,9800.000000,8005.091095,2135700.0,2.0,-3.212769,38.307115,-74.912976,12.580000,0.112170
15998,2019-12-18,TPIA.JK,248750.000000,250000.000000,248125.000000,247114.624023,22134800.0,2.0,45.007688,64.526583,113.802443,12.580000,0.112170


<a id='4'></a>
# Bagian 5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style
Proses *training* meliputi mengamati perubahan harga saham, mengambil tindakan dan perhitungan *reward*. Dengan berinteraksi dengan lingkungan pasar, agen pada akhirnya akan memperoleh strategi perdagangan yang dapat memaksimalkan imbalan.

*Environment* pasar dibangun  menggunakan OpenAI Gym yang mensimulasikan pasar saham dengan data historis pasar.

<a id='4.1'></a>
## 5.1. Memisahkan Data
Data dibagi menjadi dua bagian, yaitu bagian pelatihan dan bagian pengujian dengan keterangan sebagai berikut:

Periode data pelatihan: 2009-01-01 hingga 2017-12-31

Periode data perdagangan: 2018-01-01 hingga 2022-08-15

* **data_split** membagi data menjadi himpunan *train, test,* dan/atau *validation.*
* **len()** mengembalikan jumlah item dalam suatu objek. Namun, jika objek adalah string, maka fungsi len() akan mengembalikan jumlah karakter dalam string.
* **tail()** digunakan untuk mendapatkan n baris terakhir.

In [121]:
train = data_split(processed_full_lot, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full_lot, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

8628
1980


In [122]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2155,2017-12-28,UNVR.JK,1083000.0,1088000.0,1061000.0,962059.082031,14564500.0,3.0,222.473164,67.975738,120.660030,10.18,5.395500
2156,2017-12-29,JECC.JK,470000.0,470000.0,470000.0,434712.109375,100.0,4.0,-45.023809,43.469840,-56.271744,11.04,15.259091
2156,2017-12-29,TMAS.JK,22800.0,22900.0,22800.0,18516.827393,3500.0,4.0,-1.871566,48.538841,-32.664597,11.04,15.259091
2156,2017-12-29,TPIA.JK,143125.0,150000.0,141875.0,146239.025879,24246800.0,4.0,12.320751,63.029738,167.268652,11.04,15.259091
2156,2017-12-29,UNVR.JK,1088000.0,1119500.0,1087500.0,988586.328125,12763000.0,4.0,243.418841,72.053384,162.612829,11.04,15.259091


In [123]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,JECC.JK,470000.0,470000.0,470000.0,434712.109375,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604064
0,2018-01-02,TMAS.JK,22900.0,22900.0,21800.0,17627.371216,16500.0,1.0,-2.067101,43.270800,-110.403899,9.77,6.604064
0,2018-01-02,TPIA.JK,150625.0,150625.0,143750.0,144411.059570,22387200.0,1.0,19.810029,60.719825,142.844563,9.77,6.604064
0,2018-01-02,UNVR.JK,1123500.0,1162000.0,1100000.0,988144.335938,12108500.0,1.0,264.457087,71.890166,158.326657,9.77,6.604064
1,2018-01-03,JECC.JK,470000.0,470000.0,470000.0,434712.109375,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801895


In [124]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 4, State Space: 21


Transaction Fee
    # BUY Fee => 0.23% (Broker Fee(0.19%) + Levy(0.04%))
    # SELL Fee => 0.54% (Broker Fee(0.29%) + Levy(0.04%) + PPN(0.11%) + PPh(0.1%))

In [125]:
buy_cost_list = [0.0023] * stock_dimension
sell_cost_list = [0.0054] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

<a id='4.2'></a>
## 5.2. Lingkungan untuk Pelatihan *(Training)*



In [126]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Bagian 6: Melatih Agen DRL
* Algoritma DRL dibangun dengan menggunakan **Stable Baselines 3**. 

* FinRL mencakup algoritma DRL standar yang disempurnakan, seperti DQN, DDPG, DDPG Multi-Agen, PPO, SAC, A2C, dan TD3. FinRL juga mengizinkan pengguna untuk merancang algoritma DRL sendiri dengan mengadaptasi algoritma DRL yang telah disebutkan.



-----
kelas DRLAgent:
    Menyediakan implementasi untuk algoritma DRL

    Atribut
    ----------
        env: gym environment class
            kelas yang ditentukan pengguna

    Methods
    -------
        get_model()
            mengatur algoritma DRL
        train_model()
            melatih algoritma DRL dalam set data latih dan mengeluarkan model yang dilatih
        DRL_prediction()
            membuat prediksi dalam kumpulan data pengujian dan dapatkan hasil


In [127]:
agent = DRLAgent(env = env_train)

**Agent Training: 5 algorithms (A2C, PPO, TD3, SAC, DDPG)**

<a id='5.1'></a>
##6.1. Agen 1: A2C


In [128]:
A2C_PARAMS = {
    "n_steps": 150, 
    "ent_coef": 0.0001, 
    "learning_rate": 0.0014,
}
model_a2c = agent.get_model("a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 150, 'ent_coef': 0.0001, 'learning_rate': 0.0014}
Using cuda device


In [129]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

------------------------------------
| time/                 |          |
|    fps                | 355      |
|    iterations         | 100      |
|    time_elapsed       | 42       |
|    total_timesteps    | 15000    |
| train/                |          |
|    entropy_loss       | -5.7     |
|    explained_variance | 1.61e-06 |
|    learning_rate      | 0.0014   |
|    n_updates          | 99       |
|    policy_loss        | 6.8e+04  |
|    reward             | 0.0      |
|    std                | 1.01     |
|    value_loss         | 2.06e+08 |
------------------------------------
day: 2156, episode: 10
begin_total_asset: 100000000.00
end_total_asset: 2329200406.38
total_reward: 2229200406.38
total_cost: 794420.38
total_trades: 4765
Sharpe: 0.877
------------------------------------
| time/                 |          |
|    fps                | 364      |
|    iterations         | 200      |
|    time_elapsed       | 82       |
|    total_timesteps    | 30000    |
| train/         

<a id='5.2'></a>
##6.2. Agen 2: PPO

In [130]:
PPO_PARAMS = {
    "n_steps": 50000,
    "ent_coef": 0.7,
    "learning_rate": 0.0016,
    "batch_size": 1750,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 50000, 'ent_coef': 0.7, 'learning_rate': 0.0016, 'batch_size': 1750}
Using cuda device


/usr/local/lib/python3.7/dist-packages/stable_baselines3/ppo/ppo.py:147: UserWarning: You have specified a mini-batch size of 1750, but because the `RolloutBuffer` is of size `n_steps * n_envs = 50000`, after every 28 untruncated mini-batches, there will be a truncated mini-batch of size 1000
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=50000 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [131]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

day: 2156, episode: 30
begin_total_asset: 100000000.00
end_total_asset: 2273217.44
total_reward: -97726782.56
total_cost: 28699139.91
total_trades: 6838
Sharpe: -0.491
day: 2156, episode: 40
begin_total_asset: 100000000.00
end_total_asset: 43623686.05
total_reward: -56376313.95
total_cost: 177554839.85
total_trades: 7877
Sharpe: -0.170
---------------------------------
| time/              |          |
|    fps             | 376      |
|    iterations      | 1        |
|    time_elapsed    | 132      |
|    total_timesteps | 50000    |
| train/             |          |
|    reward          | 6.091995 |
---------------------------------


<a id='5.3'></a>
##6.3. Agen 3: TD3

In [132]:
TD3_PARAMS = {
    "batch_size": 7000, 
    "buffer_size": 10000, 
    "learning_rate": 0.9  
}
model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 7000, 'buffer_size': 10000, 'learning_rate': 0.9}
Using cuda device


In [133]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000)

day: 2156, episode: 50
begin_total_asset: 100000000.00
end_total_asset: 1248677090.02
total_reward: 1148677090.02
total_cost: 3103033.91
total_trades: 5826
Sharpe: 0.926
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 97        |
|    time_elapsed    | 88        |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | 2.06e+08  |
|    critic_loss     | 1.45e+18  |
|    learning_rate   | 0.9       |
|    n_updates       | 6471      |
|    reward          | 8097.9585 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 86        |
|    time_elapsed    | 200       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | 1.33e+08  |
|    critic_loss     | 8.62e+16  |
|    learning_rate   | 0.9       |
|    n_updates       | 15

<a id='5.4'></a>
##6.4. Agen 4: SAC

In [134]:
SAC_PARAMS = {
    "ent_coef": 1.25,
    "learning_rate": 0.0001,
    "batch_size": 4000,
    "buffer_size": 5000,
    "learning_starts": 14,
    
}
model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'ent_coef': 1.25, 'learning_rate': 0.0001, 'batch_size': 4000, 'buffer_size': 5000, 'learning_starts': 14}
Using cuda device


In [135]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000)

----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 69        |
|    time_elapsed    | 124       |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | 1.8e+04   |
|    critic_loss     | 3.9e+06   |
|    ent_coef        | 1.25      |
|    learning_rate   | 0.0001    |
|    n_updates       | 8613      |
|    reward          | 265.27246 |
----------------------------------
day: 2156, episode: 80
begin_total_asset: 100000000.00
end_total_asset: 99785700.38
total_reward: -214299.62
total_cost: 229469.40
total_trades: 6468
Sharpe: 0.449
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 68        |
|    time_elapsed    | 250       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | 1.89e+04  |
|    critic_loss     | 1.06e+06  |
|    ent_coef        | 1.25    

<a id='5.5'></a>
##6.5. Agen 5: DDPG

In [136]:
DDPG_PARAMS = {
    "batch_size": 1250, 
    "buffer_size": 50000, 
    "learning_rate": 0.0003
}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 1250, 'buffer_size': 50000, 'learning_rate': 0.0003}
Using cuda device


In [137]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=60000)

day: 2156, episode: 90
begin_total_asset: 100000000.00
end_total_asset: 99785700.38
total_reward: -214299.62
total_cost: 229469.40
total_trades: 4312
Sharpe: 0.449
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 129       |
|    time_elapsed    | 66        |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | -6.1e+03  |
|    critic_loss     | 5.42e+07  |
|    learning_rate   | 0.0003    |
|    n_updates       | 6471      |
|    reward          | 265.27246 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 116       |
|    time_elapsed    | 148       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | -1.67e+03 |
|    critic_loss     | 5.26e+06  |
|    learning_rate   | 0.0003    |
|    n_updates       | 15099   

<a id='6'></a>
# Bagian 7: Melakukan Jual-Beli

<a id='6.1'></a>
## 7.1. Performa di Dalam Sampel

Asumsikan modal awal adalah Rp10.000.000.

Pengaturan ambang turbulensi *(turbulence threshold)*

Atur ambang turbulensi menjadi lebih besar dari maksimum dalam data turbulensi sampel. Jika indeks turbulensi saat ini lebih besar dari ambang batas, maka dapat diasumsikan bahwa pasar pada periode tersebut sedang bergejolak

* **drop_duplicates()** menghapus baris duplikat.
* **quantile()** digunakan untuk menghitung kuantil nilai dalam sumbu tertentu. Sumbu default adalah baris.

In [138]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [139]:
insample_risk_indicator.vix.describe()

count    2157.000000
mean       18.666583
std         7.813415
min         9.140000
25%        13.370000
50%        16.299999
75%        21.540001
max        56.650002
Name: vix, dtype: float64

In [140]:
insample_risk_indicator.vix.quantile(0.996)

48.84047967529307

In [141]:
insample_risk_indicator.turbulence.describe()

count    2157.000000
mean        3.855361
std        11.346037
min         0.000000
25%         0.269244
50%         1.242761
75%         3.754819
max       306.792718
Name: turbulence, dtype: float64

In [142]:
insample_risk_indicator.turbulence.quantile(0.996)

50.30340851245414

<a id='6.2'></a>
##7.2. Performa di Luar Sampel

FinRL dapat diatur dengan cara melakukan pelatihan ulang secara berkala, misalnya, pelatihan ulang setiap tiga bulan, bulanan, atau mingguan. 

Pada notebook ini, *hyperparameter* hanya diatur satu kali dengan menggunakan data sampel selama periode yang telah ditetapkan. Sehingga, terjadi beberapa peluruhan alfa seiring dengan perpanjangan tanggal perdagangan.

Banyak *hyperparameter* – mis. learning rate, mempengaruhi proses pembelajaran dan biasanya ditentukan dengan menguji beberapa variasi.

In [143]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [144]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,JECC.JK,470000.0,470000.0,470000.0,434712.109375,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604064
0,2018-01-02,TMAS.JK,22900.0,22900.0,21800.0,17627.371216,16500.0,1.0,-2.067101,43.270800,-110.403899,9.77,6.604064
0,2018-01-02,TPIA.JK,150625.0,150625.0,143750.0,144411.059570,22387200.0,1.0,19.810029,60.719825,142.844563,9.77,6.604064
0,2018-01-02,UNVR.JK,1123500.0,1162000.0,1100000.0,988144.335938,12108500.0,1.0,264.457087,71.890166,158.326657,9.77,6.604064
1,2018-01-03,JECC.JK,470000.0,470000.0,470000.0,434712.109375,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801895


<a id='6.3'></a>
##7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi

<a id='6.3.1'></a>
###7.3.1. Agen 1: A2C

In [145]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym)

hit end!


In [146]:
df_actions_and_account_value_a2c = pd.merge(df_actions_a2c, df_account_value_a2c, on='date', how='outer')

#adding new column
df_actions_and_account_value_a2c['total_shares'] = ""

#count the number of column of tics
numberOfColumns = len(df_actions_and_account_value_a2c.columns)
numberOfTic = numberOfColumns-2

#selecting column of actions in every tics
action_columns_a2c = df_actions_and_account_value_a2c.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_a2c['total_shares'] = action_columns_a2c.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_a2c['total_shares'] = df_actions_and_account_value_a2c['total_shares'].cumsum(skipna=False)

In [147]:
#define function to swap columns
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

#swap points and rebounds columns
df_actions_and_account_value_a2c = swap_columns(df_actions_and_account_value_a2c, 'total_shares','account_value')

#Replace NaN value to 0
df_actions_and_account_value_a2c=df_actions_and_account_value_a2c.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_a2c)

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,100.0,0.0,53.0,0.0,153.0,1.000000e+08
1,2018-01-03,100.0,0.0,36.0,0.0,289.0,9.968864e+07
2,2018-01-04,0.0,0.0,0.0,0.0,289.0,9.908893e+07
3,2018-01-05,0.0,0.0,0.0,0.0,289.0,9.941432e+07
4,2018-01-08,0.0,0.0,0.0,0.0,289.0,1.083095e+08
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,289.0,1.453250e+08
491,2019-12-13,0.0,0.0,0.0,0.0,289.0,1.454352e+08
492,2019-12-16,0.0,0.0,0.0,0.0,289.0,1.454903e+08
493,2019-12-17,0.0,0.0,0.0,0.0,289.0,1.455455e+08


In [148]:
df_actions_and_account_value_a2c.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi4/df_actions_and_account_value_a2c(4).csv',index=False)

<a id='6.3.2'></a>
###7.3.2. Agen 2: PPO

In [149]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym)

hit end!


In [150]:
df_actions_and_account_value_ppo = pd.merge(df_actions_ppo, df_account_value_ppo, on='date', how='outer')

#adding new column
df_actions_and_account_value_ppo['total_shares'] = ""

#selecting column of actions in every tics
action_columns_ppo = df_actions_and_account_value_ppo.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_ppo['total_shares'] = action_columns_ppo.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_ppo['total_shares'] = df_actions_and_account_value_ppo['total_shares'].cumsum(skipna=False)

In [151]:
#swap points and rebounds columns
df_actions_and_account_value_ppo = swap_columns(df_actions_and_account_value_ppo, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_ppo=df_actions_and_account_value_ppo.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_ppo)

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,2.0,4.0,0.0,0.0,6.0,1.000000e+08
1,2018-01-03,2.0,4.0,0.0,0.0,12.0,1.000030e+08
2,2018-01-04,2.0,4.0,0.0,0.0,18.0,9.999566e+07
3,2018-01-05,2.0,4.0,0.0,0.0,24.0,9.999738e+07
4,2018-01-08,2.0,4.0,0.0,0.0,30.0,1.003506e+08
...,...,...,...,...,...,...,...
490,2019-12-12,-1.0,0.0,0.0,0.0,1306.0,1.143282e+08
491,2019-12-13,-1.0,0.0,0.0,0.0,1305.0,1.145093e+08
492,2019-12-16,2.0,4.0,0.0,0.0,1311.0,1.144138e+08
493,2019-12-17,-2.0,0.0,0.0,0.0,1309.0,1.144108e+08


In [152]:
df_actions_and_account_value_ppo.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi4/df_actions_and_account_value_ppo(4).csv',index=False)

<a id='6.3.3'></a>
###7.3.3. Agen 3: TD3

In [153]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym)

hit end!


In [154]:
df_actions_and_account_value_td3 = pd.merge(df_actions_td3, df_account_value_td3, on='date', how='outer')

#adding new column
df_actions_and_account_value_td3['total_shares'] = ""

#selecting column of actions in every tics
action_columns_td3 = df_actions_and_account_value_td3.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_td3['total_shares'] = action_columns_td3.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_td3['total_shares'] = df_actions_and_account_value_td3['total_shares'].cumsum(skipna=False)

In [155]:
#swap points and rebounds columns
df_actions_and_account_value_td3 = swap_columns(df_actions_and_account_value_td3, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_td3=df_actions_and_account_value_td3.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_td3)

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,0.0,0.0,100.0,0.0,100.0,1.000000e+08
1,2018-01-03,0.0,0.0,100.0,0.0,200.0,9.960119e+07
2,2018-01-04,0.0,0.0,100.0,0.0,300.0,9.847202e+07
3,2018-01-05,0.0,0.0,100.0,0.0,400.0,9.953771e+07
4,2018-01-08,0.0,0.0,100.0,0.0,500.0,9.999321e+07
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,716.0,1.747182e+08
491,2019-12-13,0.0,0.0,0.0,0.0,716.0,1.756051e+08
492,2019-12-16,0.0,0.0,0.0,0.0,716.0,1.760486e+08
493,2019-12-17,0.0,0.0,0.0,0.0,716.0,1.764920e+08


In [156]:
df_actions_and_account_value_td3.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi4/df_actions_and_account_value_td3(4).csv',index=False)

<a id='6.3.4'></a>
###7.3.4. Agen 4: SAC

In [157]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

hit end!


In [158]:
df_actions_and_account_value_sac = pd.merge(df_actions_sac, df_account_value_sac, on='date', how='outer')

#adding new column
df_actions_and_account_value_sac['total_shares'] = ""

#selecting column of actions in every tics
action_columns_sac = df_actions_and_account_value_sac.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_sac['total_shares'] = action_columns_sac.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_sac['total_shares'] = df_actions_and_account_value_sac['total_shares'].cumsum(skipna=False)

In [159]:
#swap points and rebounds columns
df_actions_and_account_value_sac = swap_columns(df_actions_and_account_value_sac, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_sac=df_actions_and_account_value_sac.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_sac)

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,0.0,54.0,0.0,100.0,154.0,1.000000e+08
1,2018-01-03,0.0,0.0,0.0,0.0,154.0,9.652446e+07
2,2018-01-04,0.0,0.0,0.0,0.0,154.0,9.542845e+07
3,2018-01-05,0.0,0.0,0.0,0.0,154.0,9.650700e+07
4,2018-01-08,0.0,0.0,0.0,0.0,154.0,9.722751e+07
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,154.0,7.659437e+07
491,2019-12-13,0.0,0.0,0.0,0.0,154.0,7.702109e+07
492,2019-12-16,0.0,0.0,0.0,0.0,154.0,7.641296e+07
493,2019-12-17,0.0,0.0,0.0,0.0,154.0,7.655229e+07


In [160]:
df_actions_and_account_value_sac.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi4/df_actions_and_account_value_sac(3).csv',index=False)

<a id='6.3.5'></a>
###7.3.5. Agen 5: DDPG

In [161]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)

hit end!


In [162]:
df_actions_and_account_value_ddpg = pd.merge(df_actions_ddpg, df_account_value_ddpg, on='date', how='outer')

#adding new column
df_actions_and_account_value_ddpg['total_shares'] = ""

#selecting column of actions in every tics
action_columns_ddpg = df_actions_and_account_value_ddpg.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_ddpg['total_shares'] = action_columns_ddpg.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_ddpg['total_shares'] = df_actions_and_account_value_ddpg['total_shares'].cumsum(skipna=False)

In [163]:
#swap points and rebounds columns
df_actions_and_account_value_ddpg = swap_columns(df_actions_and_account_value_ddpg, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_ddpg=df_actions_and_account_value_ddpg.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_ddpg)

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,0.0,54.0,0.0,100.0,154.0,1.000000e+08
1,2018-01-03,0.0,0.0,0.0,0.0,154.0,9.652446e+07
2,2018-01-04,0.0,0.0,0.0,0.0,154.0,9.542845e+07
3,2018-01-05,0.0,0.0,0.0,0.0,154.0,9.650700e+07
4,2018-01-08,0.0,0.0,0.0,0.0,154.0,9.722751e+07
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,154.0,7.659437e+07
491,2019-12-13,0.0,0.0,0.0,0.0,154.0,7.702109e+07
492,2019-12-16,0.0,0.0,0.0,0.0,154.0,7.641296e+07
493,2019-12-17,0.0,0.0,0.0,0.0,154.0,7.655229e+07


In [164]:
df_actions_and_account_value_ddpg.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi4/df_actions_and_account_value_ddpg(4).csv',index=False)

##7.4 Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi (dalam bentuk string)

###7.4.1. Agen 1: A2C

In [165]:
#Replace NaN value to 0
action_columns_a2c=action_columns_a2c.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_a2c)

j=0
for j in range(numberOfRows):
  for i in action_columns_a2c:
   val = action_columns_a2c[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_a2c, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi4/df_actions_and_account_str_value_a2c(4).csv',index=False)
df_actions_str

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,Beli 100.0 saham,Tahan,Beli 53.0 saham,Tahan,153.0,1.000000e+08
1,2018-01-03,Beli 100.0 saham,Tahan,Beli 36.0 saham,Tahan,289.0,9.968864e+07
2,2018-01-04,Tahan,Tahan,Tahan,Tahan,289.0,9.908893e+07
3,2018-01-05,Tahan,Tahan,Tahan,Tahan,289.0,9.941432e+07
4,2018-01-08,Tahan,Tahan,Tahan,Tahan,289.0,1.083095e+08
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,289.0,1.453250e+08
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,289.0,1.454352e+08
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,289.0,1.454903e+08
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,289.0,1.455455e+08


###7.4.2. Agen 2: PPO

In [166]:
#Replace NaN value to 0
action_columns_ppo=action_columns_ppo.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_ppo)

j=0
for j in range(numberOfRows):
  for i in action_columns_ppo:
   val = action_columns_ppo[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_ppo, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi4/df_actions_and_account_str_value_ppo(4).csv',index=False)
df_actions_str

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,Beli 2.0 saham,Beli 4.0 saham,Tahan,Tahan,6.0,1.000000e+08
1,2018-01-03,Beli 2.0 saham,Beli 4.0 saham,Tahan,Tahan,12.0,1.000030e+08
2,2018-01-04,Beli 2.0 saham,Beli 4.0 saham,Tahan,Tahan,18.0,9.999566e+07
3,2018-01-05,Beli 2.0 saham,Beli 4.0 saham,Tahan,Tahan,24.0,9.999738e+07
4,2018-01-08,Beli 2.0 saham,Beli 4.0 saham,Tahan,Tahan,30.0,1.003506e+08
...,...,...,...,...,...,...,...
490,2019-12-12,Jual -1.0 saham,Tahan,Tahan,Tahan,1306.0,1.143282e+08
491,2019-12-13,Jual -1.0 saham,Tahan,Tahan,Tahan,1305.0,1.145093e+08
492,2019-12-16,Beli 2.0 saham,Beli 4.0 saham,Tahan,Tahan,1311.0,1.144138e+08
493,2019-12-17,Jual -2.0 saham,Tahan,Tahan,Tahan,1309.0,1.144108e+08


###7.4.1. Agen 3: TD3

In [167]:
#Replace NaN value to 0
action_columns_td3=action_columns_td3.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_td3)

j=0
for j in range(numberOfRows):
  for i in action_columns_td3:
   val = action_columns_td3[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_td3, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi4/df_actions_and_account_str_value_td3(4).csv',index=False)
df_actions_str

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,Tahan,Tahan,Beli 100.0 saham,Tahan,100.0,1.000000e+08
1,2018-01-03,Tahan,Tahan,Beli 100.0 saham,Tahan,200.0,9.960119e+07
2,2018-01-04,Tahan,Tahan,Beli 100.0 saham,Tahan,300.0,9.847202e+07
3,2018-01-05,Tahan,Tahan,Beli 100.0 saham,Tahan,400.0,9.953771e+07
4,2018-01-08,Tahan,Tahan,Beli 100.0 saham,Tahan,500.0,9.999321e+07
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,716.0,1.747182e+08
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,716.0,1.756051e+08
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,716.0,1.760486e+08
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,716.0,1.764920e+08


###7.4.1. Agen 4: SAC

In [168]:
#Replace NaN value to 0
action_columns_sac=action_columns_sac.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_sac)

j=0
for j in range(numberOfRows):
  for i in action_columns_sac:
   val = action_columns_sac[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_sac, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi4/df_actions_and_account_str_value_sac(4).csv',index=False)
df_actions_str

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,Tahan,Beli 54.0 saham,Tahan,Beli 100.0 saham,154.0,1.000000e+08
1,2018-01-03,Tahan,Tahan,Tahan,Tahan,154.0,9.652446e+07
2,2018-01-04,Tahan,Tahan,Tahan,Tahan,154.0,9.542845e+07
3,2018-01-05,Tahan,Tahan,Tahan,Tahan,154.0,9.650700e+07
4,2018-01-08,Tahan,Tahan,Tahan,Tahan,154.0,9.722751e+07
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,154.0,7.659437e+07
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,154.0,7.702109e+07
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,154.0,7.641296e+07
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,154.0,7.655229e+07


###7.4.1. Agen 5: DDPG

In [169]:
#Replace NaN value to 0
action_columns_ddpg=action_columns_ddpg.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_ddpg)

j=0
for j in range(numberOfRows):
  for i in action_columns_ddpg:
   val = action_columns_ddpg[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_ddpg, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi4/df_actions_and_account_str_value_ddpg(4).csv',index=False)
df_actions_str

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,Tahan,Beli 54.0 saham,Tahan,Beli 100.0 saham,154.0,1.000000e+08
1,2018-01-03,Tahan,Tahan,Tahan,Tahan,154.0,9.652446e+07
2,2018-01-04,Tahan,Tahan,Tahan,Tahan,154.0,9.542845e+07
3,2018-01-05,Tahan,Tahan,Tahan,Tahan,154.0,9.650700e+07
4,2018-01-08,Tahan,Tahan,Tahan,Tahan,154.0,9.722751e+07
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,154.0,7.659437e+07
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,154.0,7.702109e+07
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,154.0,7.641296e+07
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,154.0,7.655229e+07


<a id='6.4'></a>
##7.5. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Kondisi dan Aksi

<a id='6.4.1'></a>
###7.5.1. Agen 1: A2C

In [170]:
#drop some columns
df_actions_and_account_value_a2c = df_actions_and_account_value_a2c.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_a2c:
    df_temp = df_actions_and_account_value_a2c[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic-1:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi4/1_A2C/df_states_and_actions_a2c_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604064,100.0
13140,2018-01-03,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801895,100.0
13144,2018-01-04,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333796,0.0
13148,2018-01-05,JECC.JK,4700.0,4700.0,4700.0,4347.121094,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931329,0.0
13160,2018-01-08,JECC.JK,5100.0,5175.0,5100.0,4786.458008,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,3.0,-3.619868,53.058014,-40.969081,13.94,3.749015,0.0
15976,2019-12-13,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,4.0,-3.380631,53.058014,-139.322678,12.63,0.593755,0.0
15988,2019-12-16,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,0.0,-3.154669,53.058014,-151.408554,12.14,0.376882,0.0
15992,2019-12-17,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,1.0,-2.941682,53.058014,-167.378990,12.29,0.019867,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,TMAS.JK,229.0,229.0,218.0,176.273712,16500.0,1.0,-2.067101,43.270800,-110.403899,9.77,6.604064,0.0
13141,2018-01-03,TMAS.JK,228.0,236.0,219.0,189.211227,27500.0,2.0,-1.382511,51.234612,-13.715756,9.15,24.801895,0.0
13145,2018-01-04,TMAS.JK,239.0,239.0,217.0,182.742493,15500.0,3.0,-1.346421,47.766229,-35.329556,9.22,10.333796,0.0
13149,2018-01-05,TMAS.JK,220.0,230.0,220.0,185.976868,25000.0,4.0,-1.044788,49.533335,-44.898608,9.22,3.931329,0.0
13161,2018-01-08,TMAS.JK,229.0,233.0,228.0,188.402649,2500.0,0.0,-0.603051,50.824102,28.633429,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,TMAS.JK,99.0,100.0,97.0,78.433716,3130800.0,3.0,-3.753758,35.769412,-110.875793,13.94,3.749015,0.0
15977,2019-12-13,TMAS.JK,98.0,100.0,98.0,80.050911,1143700.0,4.0,-3.607023,37.844101,-93.126705,12.63,0.593755,0.0
15989,2019-12-16,TMAS.JK,100.0,100.0,98.0,79.242325,1726300.0,0.0,-3.515456,37.222229,-89.104401,12.14,0.376882,0.0
15993,2019-12-17,TMAS.JK,99.0,99.0,97.0,79.242325,2965500.0,1.0,-3.403654,37.222229,-89.887110,12.29,0.019867,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,TPIA.JK,1506.25,1506.25,1437.50,1444.110596,22387200.0,1.0,19.810029,60.719825,142.844563,9.77,6.604064,53.0
13142,2018-01-03,TPIA.JK,1481.25,1487.50,1425.00,1407.550659,16574000.0,2.0,18.517521,56.440260,85.907381,9.15,24.801895,36.0
13146,2018-01-04,TPIA.JK,1443.75,1443.75,1375.00,1352.711060,21859600.0,3.0,12.919177,50.876154,-26.782100,9.22,10.333796,0.0
13150,2018-01-05,TPIA.JK,1387.50,1443.75,1375.00,1389.270996,27563200.0,4.0,11.302246,54.003430,1.727055,9.22,3.931329,0.0
13162,2018-01-08,TPIA.JK,1431.25,1443.75,1406.25,1401.457397,25656000.0,0.0,10.878753,54.991442,33.184555,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,TPIA.JK,2462.50,2487.50,2450.00,2440.179688,32832800.0,3.0,37.659417,62.950307,136.531575,13.94,3.749015,0.0
15978,2019-12-13,TPIA.JK,2468.75,2487.50,2462.50,2452.566406,25854400.0,4.0,40.175702,63.571498,132.220628,12.63,0.593755,0.0
15990,2019-12-16,TPIA.JK,2475.00,2493.75,2462.50,2458.759521,20370000.0,0.0,42.183346,63.884688,123.628220,12.14,0.376882,0.0
15994,2019-12-17,TPIA.JK,2481.25,2506.25,2475.00,2464.952881,20856400.0,1.0,43.769622,64.203073,122.159603,12.29,0.019867,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNVR.JK,11235.0,11620.0,11000.0,9881.443359,12108500.0,1.0,264.457087,71.890166,158.326657,9.77,6.604064,0.0
13143,2018-01-03,UNVR.JK,11175.0,11180.0,10640.0,9549.849609,13805500.0,2.0,240.790554,61.141268,83.044326,9.15,24.801895,0.0
13147,2018-01-04,UNVR.JK,10680.0,10800.0,10655.0,9443.741211,9651500.0,3.0,211.039859,58.257800,50.807092,9.22,10.333796,0.0
13151,2018-01-05,UNVR.JK,10680.0,10800.0,10635.0,9549.849609,8882500.0,4.0,193.790385,60.199544,52.000243,9.22,3.931329,0.0
13163,2018-01-08,UNVR.JK,10800.0,10940.0,10800.0,9620.590820,6800000.0,0.0,183.710576,61.436735,70.462063,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNVR.JK,8350.0,8420.0,8200.0,7616.660156,13365500.0,3.0,-49.981819,41.652679,-139.322678,13.94,3.749015,0.0
15979,2019-12-13,UNVR.JK,8280.0,8300.0,8200.0,7658.458984,12891500.0,4.0,-56.350273,42.737208,-151.408554,12.63,0.593755,0.0
15991,2019-12-16,UNVR.JK,8250.0,8275.0,8180.0,7598.082520,12503000.0,0.0,-65.513991,41.582279,-167.378990,12.14,0.376882,0.0
15995,2019-12-17,UNVR.JK,8180.0,8225.0,8180.0,7612.015625,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.29,0.019867,0.0


####String Action Version

In [171]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  #i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi4/1_A2C/df_states_and_actions_str_a2c_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,JECC.JK
0,13136,2018-01-02,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604064,100.0,Beli 100.0 saham
1,13140,2018-01-03,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801895,100.0,Beli 100.0 saham
2,13144,2018-01-04,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333796,0.0,Tahan
3,13148,2018-01-05,JECC.JK,4700.0,4700.0,4700.0,4347.121094,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931329,0.0,Tahan
4,13160,2018-01-08,JECC.JK,5100.0,5175.0,5100.0,4786.458008,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029033,0.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,3.0,-3.619868,53.058014,-40.969081,13.94,3.749015,0.0,Tahan
491,15976,2019-12-13,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,4.0,-3.380631,53.058014,-139.322678,12.63,0.593755,0.0,Tahan
492,15988,2019-12-16,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,0.0,-3.154669,53.058014,-151.408554,12.14,0.376882,0.0,Tahan
493,15992,2019-12-17,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,1.0,-2.941682,53.058014,-167.378990,12.29,0.019867,0.0,Tahan


,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,TMAS.JK
0,13137,2018-01-02,TMAS.JK,229.0,229.0,218.0,176.273712,16500.0,1.0,-2.067101,43.270800,-110.403899,9.77,6.604064,0.0,Tahan
1,13141,2018-01-03,TMAS.JK,228.0,236.0,219.0,189.211227,27500.0,2.0,-1.382511,51.234612,-13.715756,9.15,24.801895,0.0,Tahan
2,13145,2018-01-04,TMAS.JK,239.0,239.0,217.0,182.742493,15500.0,3.0,-1.346421,47.766229,-35.329556,9.22,10.333796,0.0,Tahan
3,13149,2018-01-05,TMAS.JK,220.0,230.0,220.0,185.976868,25000.0,4.0,-1.044788,49.533335,-44.898608,9.22,3.931329,0.0,Tahan
4,13161,2018-01-08,TMAS.JK,229.0,233.0,228.0,188.402649,2500.0,0.0,-0.603051,50.824102,28.633429,9.52,8.029033,0.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,TMAS.JK,99.0,100.0,97.0,78.433716,3130800.0,3.0,-3.753758,35.769412,-110.875793,13.94,3.749015,0.0,Tahan
491,15977,2019-12-13,TMAS.JK,98.0,100.0,98.0,80.050911,1143700.0,4.0,-3.607023,37.844101,-93.126705,12.63,0.593755,0.0,Tahan
492,15989,2019-12-16,TMAS.JK,100.0,100.0,98.0,79.242325,1726300.0,0.0,-3.515456,37.222229,-89.104401,12.14,0.376882,0.0,Tahan
493,15993,2019-12-17,TMAS.JK,99.0,99.0,97.0,79.242325,2965500.0,1.0,-3.403654,37.222229,-89.887110,12.29,0.019867,0.0,Tahan


,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,TPIA.JK
0,13138,2018-01-02,TPIA.JK,1506.25,1506.25,1437.50,1444.110596,22387200.0,1.0,19.810029,60.719825,142.844563,9.77,6.604064,53.0,Beli 53.0 saham
1,13142,2018-01-03,TPIA.JK,1481.25,1487.50,1425.00,1407.550659,16574000.0,2.0,18.517521,56.440260,85.907381,9.15,24.801895,36.0,Beli 36.0 saham
2,13146,2018-01-04,TPIA.JK,1443.75,1443.75,1375.00,1352.711060,21859600.0,3.0,12.919177,50.876154,-26.782100,9.22,10.333796,0.0,Tahan
3,13150,2018-01-05,TPIA.JK,1387.50,1443.75,1375.00,1389.270996,27563200.0,4.0,11.302246,54.003430,1.727055,9.22,3.931329,0.0,Tahan
4,13162,2018-01-08,TPIA.JK,1431.25,1443.75,1406.25,1401.457397,25656000.0,0.0,10.878753,54.991442,33.184555,9.52,8.029033,0.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,TPIA.JK,2462.50,2487.50,2450.00,2440.179688,32832800.0,3.0,37.659417,62.950307,136.531575,13.94,3.749015,0.0,Tahan
491,15978,2019-12-13,TPIA.JK,2468.75,2487.50,2462.50,2452.566406,25854400.0,4.0,40.175702,63.571498,132.220628,12.63,0.593755,0.0,Tahan
492,15990,2019-12-16,TPIA.JK,2475.00,2493.75,2462.50,2458.759521,20370000.0,0.0,42.183346,63.884688,123.628220,12.14,0.376882,0.0,Tahan
493,15994,2019-12-17,TPIA.JK,2481.25,2506.25,2475.00,2464.952881,20856400.0,1.0,43.769622,64.203073,122.159603,12.29,0.019867,0.0,Tahan


,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,UNVR.JK
0,13139,2018-01-02,UNVR.JK,11235.0,11620.0,11000.0,9881.443359,12108500.0,1.0,264.457087,71.890166,158.326657,9.77,6.604064,0.0,Tahan
1,13143,2018-01-03,UNVR.JK,11175.0,11180.0,10640.0,9549.849609,13805500.0,2.0,240.790554,61.141268,83.044326,9.15,24.801895,0.0,Tahan
2,13147,2018-01-04,UNVR.JK,10680.0,10800.0,10655.0,9443.741211,9651500.0,3.0,211.039859,58.257800,50.807092,9.22,10.333796,0.0,Tahan
3,13151,2018-01-05,UNVR.JK,10680.0,10800.0,10635.0,9549.849609,8882500.0,4.0,193.790385,60.199544,52.000243,9.22,3.931329,0.0,Tahan
4,13163,2018-01-08,UNVR.JK,10800.0,10940.0,10800.0,9620.590820,6800000.0,0.0,183.710576,61.436735,70.462063,9.52,8.029033,0.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,UNVR.JK,8350.0,8420.0,8200.0,7616.660156,13365500.0,3.0,-49.981819,41.652679,-139.322678,13.94,3.749015,0.0,Tahan
491,15979,2019-12-13,UNVR.JK,8280.0,8300.0,8200.0,7658.458984,12891500.0,4.0,-56.350273,42.737208,-151.408554,12.63,0.593755,0.0,Tahan
492,15991,2019-12-16,UNVR.JK,8250.0,8275.0,8180.0,7598.082520,12503000.0,0.0,-65.513991,41.582279,-167.378990,12.14,0.376882,0.0,Tahan
493,15995,2019-12-17,UNVR.JK,8180.0,8225.0,8180.0,7612.015625,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.29,0.019867,0.0,Tahan


<a id='6.4.2'></a>
###7.5.2.Agen 2: PPO

In [172]:
#drop some columns
df_actions_and_account_value_ppo = df_actions_and_account_value_ppo.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_ppo:
    df_temp = df_actions_and_account_value_ppo[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi4/2_PPO/df_states_and_actions_ppo_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604064,2.0
13140,2018-01-03,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801895,2.0
13144,2018-01-04,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333796,2.0
13148,2018-01-05,JECC.JK,4700.0,4700.0,4700.0,4347.121094,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931329,2.0
13160,2018-01-08,JECC.JK,5100.0,5175.0,5100.0,4786.458008,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029033,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,3.0,-3.619868,53.058014,-40.969081,13.94,3.749015,-1.0
15976,2019-12-13,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,4.0,-3.380631,53.058014,-139.322678,12.63,0.593755,-1.0
15988,2019-12-16,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,0.0,-3.154669,53.058014,-151.408554,12.14,0.376882,2.0
15992,2019-12-17,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,1.0,-2.941682,53.058014,-167.378990,12.29,0.019867,-2.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,TMAS.JK,229.0,229.0,218.0,176.273712,16500.0,1.0,-2.067101,43.270800,-110.403899,9.77,6.604064,4.0
13141,2018-01-03,TMAS.JK,228.0,236.0,219.0,189.211227,27500.0,2.0,-1.382511,51.234612,-13.715756,9.15,24.801895,4.0
13145,2018-01-04,TMAS.JK,239.0,239.0,217.0,182.742493,15500.0,3.0,-1.346421,47.766229,-35.329556,9.22,10.333796,4.0
13149,2018-01-05,TMAS.JK,220.0,230.0,220.0,185.976868,25000.0,4.0,-1.044788,49.533335,-44.898608,9.22,3.931329,4.0
13161,2018-01-08,TMAS.JK,229.0,233.0,228.0,188.402649,2500.0,0.0,-0.603051,50.824102,28.633429,9.52,8.029033,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,TMAS.JK,99.0,100.0,97.0,78.433716,3130800.0,3.0,-3.753758,35.769412,-110.875793,13.94,3.749015,0.0
15977,2019-12-13,TMAS.JK,98.0,100.0,98.0,80.050911,1143700.0,4.0,-3.607023,37.844101,-93.126705,12.63,0.593755,0.0
15989,2019-12-16,TMAS.JK,100.0,100.0,98.0,79.242325,1726300.0,0.0,-3.515456,37.222229,-89.104401,12.14,0.376882,4.0
15993,2019-12-17,TMAS.JK,99.0,99.0,97.0,79.242325,2965500.0,1.0,-3.403654,37.222229,-89.887110,12.29,0.019867,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,TPIA.JK,1506.25,1506.25,1437.50,1444.110596,22387200.0,1.0,19.810029,60.719825,142.844563,9.77,6.604064,0.0
13142,2018-01-03,TPIA.JK,1481.25,1487.50,1425.00,1407.550659,16574000.0,2.0,18.517521,56.440260,85.907381,9.15,24.801895,0.0
13146,2018-01-04,TPIA.JK,1443.75,1443.75,1375.00,1352.711060,21859600.0,3.0,12.919177,50.876154,-26.782100,9.22,10.333796,0.0
13150,2018-01-05,TPIA.JK,1387.50,1443.75,1375.00,1389.270996,27563200.0,4.0,11.302246,54.003430,1.727055,9.22,3.931329,0.0
13162,2018-01-08,TPIA.JK,1431.25,1443.75,1406.25,1401.457397,25656000.0,0.0,10.878753,54.991442,33.184555,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,TPIA.JK,2462.50,2487.50,2450.00,2440.179688,32832800.0,3.0,37.659417,62.950307,136.531575,13.94,3.749015,0.0
15978,2019-12-13,TPIA.JK,2468.75,2487.50,2462.50,2452.566406,25854400.0,4.0,40.175702,63.571498,132.220628,12.63,0.593755,0.0
15990,2019-12-16,TPIA.JK,2475.00,2493.75,2462.50,2458.759521,20370000.0,0.0,42.183346,63.884688,123.628220,12.14,0.376882,0.0
15994,2019-12-17,TPIA.JK,2481.25,2506.25,2475.00,2464.952881,20856400.0,1.0,43.769622,64.203073,122.159603,12.29,0.019867,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNVR.JK,11235.0,11620.0,11000.0,9881.443359,12108500.0,1.0,264.457087,71.890166,158.326657,9.77,6.604064,0.0
13143,2018-01-03,UNVR.JK,11175.0,11180.0,10640.0,9549.849609,13805500.0,2.0,240.790554,61.141268,83.044326,9.15,24.801895,0.0
13147,2018-01-04,UNVR.JK,10680.0,10800.0,10655.0,9443.741211,9651500.0,3.0,211.039859,58.257800,50.807092,9.22,10.333796,0.0
13151,2018-01-05,UNVR.JK,10680.0,10800.0,10635.0,9549.849609,8882500.0,4.0,193.790385,60.199544,52.000243,9.22,3.931329,0.0
13163,2018-01-08,UNVR.JK,10800.0,10940.0,10800.0,9620.590820,6800000.0,0.0,183.710576,61.436735,70.462063,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNVR.JK,8350.0,8420.0,8200.0,7616.660156,13365500.0,3.0,-49.981819,41.652679,-139.322678,13.94,3.749015,0.0
15979,2019-12-13,UNVR.JK,8280.0,8300.0,8200.0,7658.458984,12891500.0,4.0,-56.350273,42.737208,-151.408554,12.63,0.593755,0.0
15991,2019-12-16,UNVR.JK,8250.0,8275.0,8180.0,7598.082520,12503000.0,0.0,-65.513991,41.582279,-167.378990,12.14,0.376882,0.0
15995,2019-12-17,UNVR.JK,8180.0,8225.0,8180.0,7612.015625,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.29,0.019867,0.0


####String Action Version

In [173]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi4/2_PPO/df_states_and_actions_str_ppo_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,JECC.JK
0,13136,2018-01-02,4700.0,4700.0,4700.0,4347.121094,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604064,Beli 2.0 saham
1,13140,2018-01-03,4700.0,4700.0,4700.0,4347.121094,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801895,Beli 2.0 saham
2,13144,2018-01-04,4700.0,4700.0,4700.0,4347.121094,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333796,Beli 2.0 saham
3,13148,2018-01-05,4700.0,4700.0,4700.0,4347.121094,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931329,Beli 2.0 saham
4,13160,2018-01-08,5100.0,5175.0,5100.0,4786.458008,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029033,Beli 2.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,6175.0,6175.0,6175.0,6175.000000,0.0,3.0,-3.619868,53.058014,-40.969081,13.94,3.749015,Tahan
491,15976,2019-12-13,6175.0,6175.0,6175.0,6175.000000,0.0,4.0,-3.380631,53.058014,-139.322678,12.63,0.593755,Tahan
492,15988,2019-12-16,6175.0,6175.0,6175.0,6175.000000,0.0,0.0,-3.154669,53.058014,-151.408554,12.14,0.376882,Beli 2.0 saham
493,15992,2019-12-17,6175.0,6175.0,6175.0,6175.000000,0.0,1.0,-2.941682,53.058014,-167.378990,12.29,0.019867,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,TMAS.JK
0,13137,2018-01-02,229.0,229.0,218.0,176.273712,16500.0,1.0,-2.067101,43.270800,-110.403899,9.77,6.604064,Beli 4.0 saham
1,13141,2018-01-03,228.0,236.0,219.0,189.211227,27500.0,2.0,-1.382511,51.234612,-13.715756,9.15,24.801895,Beli 4.0 saham
2,13145,2018-01-04,239.0,239.0,217.0,182.742493,15500.0,3.0,-1.346421,47.766229,-35.329556,9.22,10.333796,Beli 4.0 saham
3,13149,2018-01-05,220.0,230.0,220.0,185.976868,25000.0,4.0,-1.044788,49.533335,-44.898608,9.22,3.931329,Beli 4.0 saham
4,13161,2018-01-08,229.0,233.0,228.0,188.402649,2500.0,0.0,-0.603051,50.824102,28.633429,9.52,8.029033,Beli 4.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,99.0,100.0,97.0,78.433716,3130800.0,3.0,-3.753758,35.769412,-110.875793,13.94,3.749015,Tahan
491,15977,2019-12-13,98.0,100.0,98.0,80.050911,1143700.0,4.0,-3.607023,37.844101,-93.126705,12.63,0.593755,Tahan
492,15989,2019-12-16,100.0,100.0,98.0,79.242325,1726300.0,0.0,-3.515456,37.222229,-89.104401,12.14,0.376882,Beli 4.0 saham
493,15993,2019-12-17,99.0,99.0,97.0,79.242325,2965500.0,1.0,-3.403654,37.222229,-89.887110,12.29,0.019867,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,TPIA.JK
0,13138,2018-01-02,1506.25,1506.25,1437.50,1444.110596,22387200.0,1.0,19.810029,60.719825,142.844563,9.77,6.604064,Tahan
1,13142,2018-01-03,1481.25,1487.50,1425.00,1407.550659,16574000.0,2.0,18.517521,56.440260,85.907381,9.15,24.801895,Tahan
2,13146,2018-01-04,1443.75,1443.75,1375.00,1352.711060,21859600.0,3.0,12.919177,50.876154,-26.782100,9.22,10.333796,Tahan
3,13150,2018-01-05,1387.50,1443.75,1375.00,1389.270996,27563200.0,4.0,11.302246,54.003430,1.727055,9.22,3.931329,Tahan
4,13162,2018-01-08,1431.25,1443.75,1406.25,1401.457397,25656000.0,0.0,10.878753,54.991442,33.184555,9.52,8.029033,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,2462.50,2487.50,2450.00,2440.179688,32832800.0,3.0,37.659417,62.950307,136.531575,13.94,3.749015,Tahan
491,15978,2019-12-13,2468.75,2487.50,2462.50,2452.566406,25854400.0,4.0,40.175702,63.571498,132.220628,12.63,0.593755,Tahan
492,15990,2019-12-16,2475.00,2493.75,2462.50,2458.759521,20370000.0,0.0,42.183346,63.884688,123.628220,12.14,0.376882,Tahan
493,15994,2019-12-17,2481.25,2506.25,2475.00,2464.952881,20856400.0,1.0,43.769622,64.203073,122.159603,12.29,0.019867,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,UNVR.JK
0,13139,2018-01-02,11235.0,11620.0,11000.0,9881.443359,12108500.0,1.0,264.457087,71.890166,158.326657,9.77,6.604064,Tahan
1,13143,2018-01-03,11175.0,11180.0,10640.0,9549.849609,13805500.0,2.0,240.790554,61.141268,83.044326,9.15,24.801895,Tahan
2,13147,2018-01-04,10680.0,10800.0,10655.0,9443.741211,9651500.0,3.0,211.039859,58.257800,50.807092,9.22,10.333796,Tahan
3,13151,2018-01-05,10680.0,10800.0,10635.0,9549.849609,8882500.0,4.0,193.790385,60.199544,52.000243,9.22,3.931329,Tahan
4,13163,2018-01-08,10800.0,10940.0,10800.0,9620.590820,6800000.0,0.0,183.710576,61.436735,70.462063,9.52,8.029033,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,8350.0,8420.0,8200.0,7616.660156,13365500.0,3.0,-49.981819,41.652679,-139.322678,13.94,3.749015,Tahan
491,15979,2019-12-13,8280.0,8300.0,8200.0,7658.458984,12891500.0,4.0,-56.350273,42.737208,-151.408554,12.63,0.593755,Tahan
492,15991,2019-12-16,8250.0,8275.0,8180.0,7598.082520,12503000.0,0.0,-65.513991,41.582279,-167.378990,12.14,0.376882,Tahan
493,15995,2019-12-17,8180.0,8225.0,8180.0,7612.015625,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.29,0.019867,Tahan


<a id='6.4.3'></a>
###7.5.3. Agen 3: TD3

In [174]:
#drop some columns
df_actions_and_account_value_td3 = df_actions_and_account_value_td3.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_td3:
    df_temp = df_actions_and_account_value_td3[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi4/3_TD3/df_states_and_actions_td3_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604064,0.0
13140,2018-01-03,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801895,0.0
13144,2018-01-04,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333796,0.0
13148,2018-01-05,JECC.JK,4700.0,4700.0,4700.0,4347.121094,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931329,0.0
13160,2018-01-08,JECC.JK,5100.0,5175.0,5100.0,4786.458008,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,3.0,-3.619868,53.058014,-40.969081,13.94,3.749015,0.0
15976,2019-12-13,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,4.0,-3.380631,53.058014,-139.322678,12.63,0.593755,0.0
15988,2019-12-16,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,0.0,-3.154669,53.058014,-151.408554,12.14,0.376882,0.0
15992,2019-12-17,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,1.0,-2.941682,53.058014,-167.378990,12.29,0.019867,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,TMAS.JK,229.0,229.0,218.0,176.273712,16500.0,1.0,-2.067101,43.270800,-110.403899,9.77,6.604064,0.0
13141,2018-01-03,TMAS.JK,228.0,236.0,219.0,189.211227,27500.0,2.0,-1.382511,51.234612,-13.715756,9.15,24.801895,0.0
13145,2018-01-04,TMAS.JK,239.0,239.0,217.0,182.742493,15500.0,3.0,-1.346421,47.766229,-35.329556,9.22,10.333796,0.0
13149,2018-01-05,TMAS.JK,220.0,230.0,220.0,185.976868,25000.0,4.0,-1.044788,49.533335,-44.898608,9.22,3.931329,0.0
13161,2018-01-08,TMAS.JK,229.0,233.0,228.0,188.402649,2500.0,0.0,-0.603051,50.824102,28.633429,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,TMAS.JK,99.0,100.0,97.0,78.433716,3130800.0,3.0,-3.753758,35.769412,-110.875793,13.94,3.749015,0.0
15977,2019-12-13,TMAS.JK,98.0,100.0,98.0,80.050911,1143700.0,4.0,-3.607023,37.844101,-93.126705,12.63,0.593755,0.0
15989,2019-12-16,TMAS.JK,100.0,100.0,98.0,79.242325,1726300.0,0.0,-3.515456,37.222229,-89.104401,12.14,0.376882,0.0
15993,2019-12-17,TMAS.JK,99.0,99.0,97.0,79.242325,2965500.0,1.0,-3.403654,37.222229,-89.887110,12.29,0.019867,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,TPIA.JK,1506.25,1506.25,1437.50,1444.110596,22387200.0,1.0,19.810029,60.719825,142.844563,9.77,6.604064,100.0
13142,2018-01-03,TPIA.JK,1481.25,1487.50,1425.00,1407.550659,16574000.0,2.0,18.517521,56.440260,85.907381,9.15,24.801895,100.0
13146,2018-01-04,TPIA.JK,1443.75,1443.75,1375.00,1352.711060,21859600.0,3.0,12.919177,50.876154,-26.782100,9.22,10.333796,100.0
13150,2018-01-05,TPIA.JK,1387.50,1443.75,1375.00,1389.270996,27563200.0,4.0,11.302246,54.003430,1.727055,9.22,3.931329,100.0
13162,2018-01-08,TPIA.JK,1431.25,1443.75,1406.25,1401.457397,25656000.0,0.0,10.878753,54.991442,33.184555,9.52,8.029033,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,TPIA.JK,2462.50,2487.50,2450.00,2440.179688,32832800.0,3.0,37.659417,62.950307,136.531575,13.94,3.749015,0.0
15978,2019-12-13,TPIA.JK,2468.75,2487.50,2462.50,2452.566406,25854400.0,4.0,40.175702,63.571498,132.220628,12.63,0.593755,0.0
15990,2019-12-16,TPIA.JK,2475.00,2493.75,2462.50,2458.759521,20370000.0,0.0,42.183346,63.884688,123.628220,12.14,0.376882,0.0
15994,2019-12-17,TPIA.JK,2481.25,2506.25,2475.00,2464.952881,20856400.0,1.0,43.769622,64.203073,122.159603,12.29,0.019867,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNVR.JK,11235.0,11620.0,11000.0,9881.443359,12108500.0,1.0,264.457087,71.890166,158.326657,9.77,6.604064,0.0
13143,2018-01-03,UNVR.JK,11175.0,11180.0,10640.0,9549.849609,13805500.0,2.0,240.790554,61.141268,83.044326,9.15,24.801895,0.0
13147,2018-01-04,UNVR.JK,10680.0,10800.0,10655.0,9443.741211,9651500.0,3.0,211.039859,58.257800,50.807092,9.22,10.333796,0.0
13151,2018-01-05,UNVR.JK,10680.0,10800.0,10635.0,9549.849609,8882500.0,4.0,193.790385,60.199544,52.000243,9.22,3.931329,0.0
13163,2018-01-08,UNVR.JK,10800.0,10940.0,10800.0,9620.590820,6800000.0,0.0,183.710576,61.436735,70.462063,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNVR.JK,8350.0,8420.0,8200.0,7616.660156,13365500.0,3.0,-49.981819,41.652679,-139.322678,13.94,3.749015,0.0
15979,2019-12-13,UNVR.JK,8280.0,8300.0,8200.0,7658.458984,12891500.0,4.0,-56.350273,42.737208,-151.408554,12.63,0.593755,0.0
15991,2019-12-16,UNVR.JK,8250.0,8275.0,8180.0,7598.082520,12503000.0,0.0,-65.513991,41.582279,-167.378990,12.14,0.376882,0.0
15995,2019-12-17,UNVR.JK,8180.0,8225.0,8180.0,7612.015625,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.29,0.019867,0.0


####String Action Version

In [175]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi4/3_TD3/df_states_and_actions_str_td3_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,JECC.JK
0,13136,2018-01-02,4700.0,4700.0,4700.0,4347.121094,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604064,Tahan
1,13140,2018-01-03,4700.0,4700.0,4700.0,4347.121094,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801895,Tahan
2,13144,2018-01-04,4700.0,4700.0,4700.0,4347.121094,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333796,Tahan
3,13148,2018-01-05,4700.0,4700.0,4700.0,4347.121094,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931329,Tahan
4,13160,2018-01-08,5100.0,5175.0,5100.0,4786.458008,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029033,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,6175.0,6175.0,6175.0,6175.000000,0.0,3.0,-3.619868,53.058014,-40.969081,13.94,3.749015,Tahan
491,15976,2019-12-13,6175.0,6175.0,6175.0,6175.000000,0.0,4.0,-3.380631,53.058014,-139.322678,12.63,0.593755,Tahan
492,15988,2019-12-16,6175.0,6175.0,6175.0,6175.000000,0.0,0.0,-3.154669,53.058014,-151.408554,12.14,0.376882,Tahan
493,15992,2019-12-17,6175.0,6175.0,6175.0,6175.000000,0.0,1.0,-2.941682,53.058014,-167.378990,12.29,0.019867,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,TMAS.JK
0,13137,2018-01-02,229.0,229.0,218.0,176.273712,16500.0,1.0,-2.067101,43.270800,-110.403899,9.77,6.604064,Tahan
1,13141,2018-01-03,228.0,236.0,219.0,189.211227,27500.0,2.0,-1.382511,51.234612,-13.715756,9.15,24.801895,Tahan
2,13145,2018-01-04,239.0,239.0,217.0,182.742493,15500.0,3.0,-1.346421,47.766229,-35.329556,9.22,10.333796,Tahan
3,13149,2018-01-05,220.0,230.0,220.0,185.976868,25000.0,4.0,-1.044788,49.533335,-44.898608,9.22,3.931329,Tahan
4,13161,2018-01-08,229.0,233.0,228.0,188.402649,2500.0,0.0,-0.603051,50.824102,28.633429,9.52,8.029033,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,99.0,100.0,97.0,78.433716,3130800.0,3.0,-3.753758,35.769412,-110.875793,13.94,3.749015,Tahan
491,15977,2019-12-13,98.0,100.0,98.0,80.050911,1143700.0,4.0,-3.607023,37.844101,-93.126705,12.63,0.593755,Tahan
492,15989,2019-12-16,100.0,100.0,98.0,79.242325,1726300.0,0.0,-3.515456,37.222229,-89.104401,12.14,0.376882,Tahan
493,15993,2019-12-17,99.0,99.0,97.0,79.242325,2965500.0,1.0,-3.403654,37.222229,-89.887110,12.29,0.019867,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,TPIA.JK
0,13138,2018-01-02,1506.25,1506.25,1437.50,1444.110596,22387200.0,1.0,19.810029,60.719825,142.844563,9.77,6.604064,Beli 100.0 saham
1,13142,2018-01-03,1481.25,1487.50,1425.00,1407.550659,16574000.0,2.0,18.517521,56.440260,85.907381,9.15,24.801895,Beli 100.0 saham
2,13146,2018-01-04,1443.75,1443.75,1375.00,1352.711060,21859600.0,3.0,12.919177,50.876154,-26.782100,9.22,10.333796,Beli 100.0 saham
3,13150,2018-01-05,1387.50,1443.75,1375.00,1389.270996,27563200.0,4.0,11.302246,54.003430,1.727055,9.22,3.931329,Beli 100.0 saham
4,13162,2018-01-08,1431.25,1443.75,1406.25,1401.457397,25656000.0,0.0,10.878753,54.991442,33.184555,9.52,8.029033,Beli 100.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,2462.50,2487.50,2450.00,2440.179688,32832800.0,3.0,37.659417,62.950307,136.531575,13.94,3.749015,Tahan
491,15978,2019-12-13,2468.75,2487.50,2462.50,2452.566406,25854400.0,4.0,40.175702,63.571498,132.220628,12.63,0.593755,Tahan
492,15990,2019-12-16,2475.00,2493.75,2462.50,2458.759521,20370000.0,0.0,42.183346,63.884688,123.628220,12.14,0.376882,Tahan
493,15994,2019-12-17,2481.25,2506.25,2475.00,2464.952881,20856400.0,1.0,43.769622,64.203073,122.159603,12.29,0.019867,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,UNVR.JK
0,13139,2018-01-02,11235.0,11620.0,11000.0,9881.443359,12108500.0,1.0,264.457087,71.890166,158.326657,9.77,6.604064,Tahan
1,13143,2018-01-03,11175.0,11180.0,10640.0,9549.849609,13805500.0,2.0,240.790554,61.141268,83.044326,9.15,24.801895,Tahan
2,13147,2018-01-04,10680.0,10800.0,10655.0,9443.741211,9651500.0,3.0,211.039859,58.257800,50.807092,9.22,10.333796,Tahan
3,13151,2018-01-05,10680.0,10800.0,10635.0,9549.849609,8882500.0,4.0,193.790385,60.199544,52.000243,9.22,3.931329,Tahan
4,13163,2018-01-08,10800.0,10940.0,10800.0,9620.590820,6800000.0,0.0,183.710576,61.436735,70.462063,9.52,8.029033,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,8350.0,8420.0,8200.0,7616.660156,13365500.0,3.0,-49.981819,41.652679,-139.322678,13.94,3.749015,Tahan
491,15979,2019-12-13,8280.0,8300.0,8200.0,7658.458984,12891500.0,4.0,-56.350273,42.737208,-151.408554,12.63,0.593755,Tahan
492,15991,2019-12-16,8250.0,8275.0,8180.0,7598.082520,12503000.0,0.0,-65.513991,41.582279,-167.378990,12.14,0.376882,Tahan
493,15995,2019-12-17,8180.0,8225.0,8180.0,7612.015625,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.29,0.019867,Tahan


<a id='6.4.4'></a>
###7.5.4. Agen 4: SAC

In [176]:
#drop some columns
df_actions_and_account_value_sac = df_actions_and_account_value_sac.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_sac:
    df_temp = df_actions_and_account_value_sac[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi4/4_SAC/df_states_and_actions_sac_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604064,0.0
13140,2018-01-03,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801895,0.0
13144,2018-01-04,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333796,0.0
13148,2018-01-05,JECC.JK,4700.0,4700.0,4700.0,4347.121094,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931329,0.0
13160,2018-01-08,JECC.JK,5100.0,5175.0,5100.0,4786.458008,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,3.0,-3.619868,53.058014,-40.969081,13.94,3.749015,0.0
15976,2019-12-13,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,4.0,-3.380631,53.058014,-139.322678,12.63,0.593755,0.0
15988,2019-12-16,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,0.0,-3.154669,53.058014,-151.408554,12.14,0.376882,0.0
15992,2019-12-17,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,1.0,-2.941682,53.058014,-167.378990,12.29,0.019867,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,TMAS.JK,229.0,229.0,218.0,176.273712,16500.0,1.0,-2.067101,43.270800,-110.403899,9.77,6.604064,54.0
13141,2018-01-03,TMAS.JK,228.0,236.0,219.0,189.211227,27500.0,2.0,-1.382511,51.234612,-13.715756,9.15,24.801895,0.0
13145,2018-01-04,TMAS.JK,239.0,239.0,217.0,182.742493,15500.0,3.0,-1.346421,47.766229,-35.329556,9.22,10.333796,0.0
13149,2018-01-05,TMAS.JK,220.0,230.0,220.0,185.976868,25000.0,4.0,-1.044788,49.533335,-44.898608,9.22,3.931329,0.0
13161,2018-01-08,TMAS.JK,229.0,233.0,228.0,188.402649,2500.0,0.0,-0.603051,50.824102,28.633429,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,TMAS.JK,99.0,100.0,97.0,78.433716,3130800.0,3.0,-3.753758,35.769412,-110.875793,13.94,3.749015,0.0
15977,2019-12-13,TMAS.JK,98.0,100.0,98.0,80.050911,1143700.0,4.0,-3.607023,37.844101,-93.126705,12.63,0.593755,0.0
15989,2019-12-16,TMAS.JK,100.0,100.0,98.0,79.242325,1726300.0,0.0,-3.515456,37.222229,-89.104401,12.14,0.376882,0.0
15993,2019-12-17,TMAS.JK,99.0,99.0,97.0,79.242325,2965500.0,1.0,-3.403654,37.222229,-89.887110,12.29,0.019867,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,TPIA.JK,1506.25,1506.25,1437.50,1444.110596,22387200.0,1.0,19.810029,60.719825,142.844563,9.77,6.604064,0.0
13142,2018-01-03,TPIA.JK,1481.25,1487.50,1425.00,1407.550659,16574000.0,2.0,18.517521,56.440260,85.907381,9.15,24.801895,0.0
13146,2018-01-04,TPIA.JK,1443.75,1443.75,1375.00,1352.711060,21859600.0,3.0,12.919177,50.876154,-26.782100,9.22,10.333796,0.0
13150,2018-01-05,TPIA.JK,1387.50,1443.75,1375.00,1389.270996,27563200.0,4.0,11.302246,54.003430,1.727055,9.22,3.931329,0.0
13162,2018-01-08,TPIA.JK,1431.25,1443.75,1406.25,1401.457397,25656000.0,0.0,10.878753,54.991442,33.184555,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,TPIA.JK,2462.50,2487.50,2450.00,2440.179688,32832800.0,3.0,37.659417,62.950307,136.531575,13.94,3.749015,0.0
15978,2019-12-13,TPIA.JK,2468.75,2487.50,2462.50,2452.566406,25854400.0,4.0,40.175702,63.571498,132.220628,12.63,0.593755,0.0
15990,2019-12-16,TPIA.JK,2475.00,2493.75,2462.50,2458.759521,20370000.0,0.0,42.183346,63.884688,123.628220,12.14,0.376882,0.0
15994,2019-12-17,TPIA.JK,2481.25,2506.25,2475.00,2464.952881,20856400.0,1.0,43.769622,64.203073,122.159603,12.29,0.019867,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNVR.JK,11235.0,11620.0,11000.0,9881.443359,12108500.0,1.0,264.457087,71.890166,158.326657,9.77,6.604064,100.0
13143,2018-01-03,UNVR.JK,11175.0,11180.0,10640.0,9549.849609,13805500.0,2.0,240.790554,61.141268,83.044326,9.15,24.801895,0.0
13147,2018-01-04,UNVR.JK,10680.0,10800.0,10655.0,9443.741211,9651500.0,3.0,211.039859,58.257800,50.807092,9.22,10.333796,0.0
13151,2018-01-05,UNVR.JK,10680.0,10800.0,10635.0,9549.849609,8882500.0,4.0,193.790385,60.199544,52.000243,9.22,3.931329,0.0
13163,2018-01-08,UNVR.JK,10800.0,10940.0,10800.0,9620.590820,6800000.0,0.0,183.710576,61.436735,70.462063,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNVR.JK,8350.0,8420.0,8200.0,7616.660156,13365500.0,3.0,-49.981819,41.652679,-139.322678,13.94,3.749015,0.0
15979,2019-12-13,UNVR.JK,8280.0,8300.0,8200.0,7658.458984,12891500.0,4.0,-56.350273,42.737208,-151.408554,12.63,0.593755,0.0
15991,2019-12-16,UNVR.JK,8250.0,8275.0,8180.0,7598.082520,12503000.0,0.0,-65.513991,41.582279,-167.378990,12.14,0.376882,0.0
15995,2019-12-17,UNVR.JK,8180.0,8225.0,8180.0,7612.015625,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.29,0.019867,0.0


####String Action Version

In [177]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi4/4_SAC/df_states_and_actions_str_sac_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,JECC.JK
0,13136,2018-01-02,4700.0,4700.0,4700.0,4347.121094,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604064,Tahan
1,13140,2018-01-03,4700.0,4700.0,4700.0,4347.121094,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801895,Tahan
2,13144,2018-01-04,4700.0,4700.0,4700.0,4347.121094,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333796,Tahan
3,13148,2018-01-05,4700.0,4700.0,4700.0,4347.121094,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931329,Tahan
4,13160,2018-01-08,5100.0,5175.0,5100.0,4786.458008,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029033,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,6175.0,6175.0,6175.0,6175.000000,0.0,3.0,-3.619868,53.058014,-40.969081,13.94,3.749015,Tahan
491,15976,2019-12-13,6175.0,6175.0,6175.0,6175.000000,0.0,4.0,-3.380631,53.058014,-139.322678,12.63,0.593755,Tahan
492,15988,2019-12-16,6175.0,6175.0,6175.0,6175.000000,0.0,0.0,-3.154669,53.058014,-151.408554,12.14,0.376882,Tahan
493,15992,2019-12-17,6175.0,6175.0,6175.0,6175.000000,0.0,1.0,-2.941682,53.058014,-167.378990,12.29,0.019867,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,TMAS.JK
0,13137,2018-01-02,229.0,229.0,218.0,176.273712,16500.0,1.0,-2.067101,43.270800,-110.403899,9.77,6.604064,Beli 54.0 saham
1,13141,2018-01-03,228.0,236.0,219.0,189.211227,27500.0,2.0,-1.382511,51.234612,-13.715756,9.15,24.801895,Tahan
2,13145,2018-01-04,239.0,239.0,217.0,182.742493,15500.0,3.0,-1.346421,47.766229,-35.329556,9.22,10.333796,Tahan
3,13149,2018-01-05,220.0,230.0,220.0,185.976868,25000.0,4.0,-1.044788,49.533335,-44.898608,9.22,3.931329,Tahan
4,13161,2018-01-08,229.0,233.0,228.0,188.402649,2500.0,0.0,-0.603051,50.824102,28.633429,9.52,8.029033,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,99.0,100.0,97.0,78.433716,3130800.0,3.0,-3.753758,35.769412,-110.875793,13.94,3.749015,Tahan
491,15977,2019-12-13,98.0,100.0,98.0,80.050911,1143700.0,4.0,-3.607023,37.844101,-93.126705,12.63,0.593755,Tahan
492,15989,2019-12-16,100.0,100.0,98.0,79.242325,1726300.0,0.0,-3.515456,37.222229,-89.104401,12.14,0.376882,Tahan
493,15993,2019-12-17,99.0,99.0,97.0,79.242325,2965500.0,1.0,-3.403654,37.222229,-89.887110,12.29,0.019867,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,TPIA.JK
0,13138,2018-01-02,1506.25,1506.25,1437.50,1444.110596,22387200.0,1.0,19.810029,60.719825,142.844563,9.77,6.604064,Tahan
1,13142,2018-01-03,1481.25,1487.50,1425.00,1407.550659,16574000.0,2.0,18.517521,56.440260,85.907381,9.15,24.801895,Tahan
2,13146,2018-01-04,1443.75,1443.75,1375.00,1352.711060,21859600.0,3.0,12.919177,50.876154,-26.782100,9.22,10.333796,Tahan
3,13150,2018-01-05,1387.50,1443.75,1375.00,1389.270996,27563200.0,4.0,11.302246,54.003430,1.727055,9.22,3.931329,Tahan
4,13162,2018-01-08,1431.25,1443.75,1406.25,1401.457397,25656000.0,0.0,10.878753,54.991442,33.184555,9.52,8.029033,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,2462.50,2487.50,2450.00,2440.179688,32832800.0,3.0,37.659417,62.950307,136.531575,13.94,3.749015,Tahan
491,15978,2019-12-13,2468.75,2487.50,2462.50,2452.566406,25854400.0,4.0,40.175702,63.571498,132.220628,12.63,0.593755,Tahan
492,15990,2019-12-16,2475.00,2493.75,2462.50,2458.759521,20370000.0,0.0,42.183346,63.884688,123.628220,12.14,0.376882,Tahan
493,15994,2019-12-17,2481.25,2506.25,2475.00,2464.952881,20856400.0,1.0,43.769622,64.203073,122.159603,12.29,0.019867,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,UNVR.JK
0,13139,2018-01-02,11235.0,11620.0,11000.0,9881.443359,12108500.0,1.0,264.457087,71.890166,158.326657,9.77,6.604064,Beli 100.0 saham
1,13143,2018-01-03,11175.0,11180.0,10640.0,9549.849609,13805500.0,2.0,240.790554,61.141268,83.044326,9.15,24.801895,Tahan
2,13147,2018-01-04,10680.0,10800.0,10655.0,9443.741211,9651500.0,3.0,211.039859,58.257800,50.807092,9.22,10.333796,Tahan
3,13151,2018-01-05,10680.0,10800.0,10635.0,9549.849609,8882500.0,4.0,193.790385,60.199544,52.000243,9.22,3.931329,Tahan
4,13163,2018-01-08,10800.0,10940.0,10800.0,9620.590820,6800000.0,0.0,183.710576,61.436735,70.462063,9.52,8.029033,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,8350.0,8420.0,8200.0,7616.660156,13365500.0,3.0,-49.981819,41.652679,-139.322678,13.94,3.749015,Tahan
491,15979,2019-12-13,8280.0,8300.0,8200.0,7658.458984,12891500.0,4.0,-56.350273,42.737208,-151.408554,12.63,0.593755,Tahan
492,15991,2019-12-16,8250.0,8275.0,8180.0,7598.082520,12503000.0,0.0,-65.513991,41.582279,-167.378990,12.14,0.376882,Tahan
493,15995,2019-12-17,8180.0,8225.0,8180.0,7612.015625,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.29,0.019867,Tahan


<a id='6.5.5'></a>
###7.4.5. Agen 5: DDPG

In [178]:
#drop some columns
df_actions_and_account_value_ddpg = df_actions_and_account_value_ddpg.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_ddpg:
    df_temp = df_actions_and_account_value_ddpg[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi4/5_DDPG/df_states_and_actions_ddpg_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604064,0.0
13140,2018-01-03,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801895,0.0
13144,2018-01-04,JECC.JK,4700.0,4700.0,4700.0,4347.121094,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333796,0.0
13148,2018-01-05,JECC.JK,4700.0,4700.0,4700.0,4347.121094,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931329,0.0
13160,2018-01-08,JECC.JK,5100.0,5175.0,5100.0,4786.458008,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,3.0,-3.619868,53.058014,-40.969081,13.94,3.749015,0.0
15976,2019-12-13,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,4.0,-3.380631,53.058014,-139.322678,12.63,0.593755,0.0
15988,2019-12-16,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,0.0,-3.154669,53.058014,-151.408554,12.14,0.376882,0.0
15992,2019-12-17,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,1.0,-2.941682,53.058014,-167.378990,12.29,0.019867,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,TMAS.JK,229.0,229.0,218.0,176.273712,16500.0,1.0,-2.067101,43.270800,-110.403899,9.77,6.604064,54.0
13141,2018-01-03,TMAS.JK,228.0,236.0,219.0,189.211227,27500.0,2.0,-1.382511,51.234612,-13.715756,9.15,24.801895,0.0
13145,2018-01-04,TMAS.JK,239.0,239.0,217.0,182.742493,15500.0,3.0,-1.346421,47.766229,-35.329556,9.22,10.333796,0.0
13149,2018-01-05,TMAS.JK,220.0,230.0,220.0,185.976868,25000.0,4.0,-1.044788,49.533335,-44.898608,9.22,3.931329,0.0
13161,2018-01-08,TMAS.JK,229.0,233.0,228.0,188.402649,2500.0,0.0,-0.603051,50.824102,28.633429,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,TMAS.JK,99.0,100.0,97.0,78.433716,3130800.0,3.0,-3.753758,35.769412,-110.875793,13.94,3.749015,0.0
15977,2019-12-13,TMAS.JK,98.0,100.0,98.0,80.050911,1143700.0,4.0,-3.607023,37.844101,-93.126705,12.63,0.593755,0.0
15989,2019-12-16,TMAS.JK,100.0,100.0,98.0,79.242325,1726300.0,0.0,-3.515456,37.222229,-89.104401,12.14,0.376882,0.0
15993,2019-12-17,TMAS.JK,99.0,99.0,97.0,79.242325,2965500.0,1.0,-3.403654,37.222229,-89.887110,12.29,0.019867,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,TPIA.JK,1506.25,1506.25,1437.50,1444.110596,22387200.0,1.0,19.810029,60.719825,142.844563,9.77,6.604064,0.0
13142,2018-01-03,TPIA.JK,1481.25,1487.50,1425.00,1407.550659,16574000.0,2.0,18.517521,56.440260,85.907381,9.15,24.801895,0.0
13146,2018-01-04,TPIA.JK,1443.75,1443.75,1375.00,1352.711060,21859600.0,3.0,12.919177,50.876154,-26.782100,9.22,10.333796,0.0
13150,2018-01-05,TPIA.JK,1387.50,1443.75,1375.00,1389.270996,27563200.0,4.0,11.302246,54.003430,1.727055,9.22,3.931329,0.0
13162,2018-01-08,TPIA.JK,1431.25,1443.75,1406.25,1401.457397,25656000.0,0.0,10.878753,54.991442,33.184555,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,TPIA.JK,2462.50,2487.50,2450.00,2440.179688,32832800.0,3.0,37.659417,62.950307,136.531575,13.94,3.749015,0.0
15978,2019-12-13,TPIA.JK,2468.75,2487.50,2462.50,2452.566406,25854400.0,4.0,40.175702,63.571498,132.220628,12.63,0.593755,0.0
15990,2019-12-16,TPIA.JK,2475.00,2493.75,2462.50,2458.759521,20370000.0,0.0,42.183346,63.884688,123.628220,12.14,0.376882,0.0
15994,2019-12-17,TPIA.JK,2481.25,2506.25,2475.00,2464.952881,20856400.0,1.0,43.769622,64.203073,122.159603,12.29,0.019867,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNVR.JK,11235.0,11620.0,11000.0,9881.443359,12108500.0,1.0,264.457087,71.890166,158.326657,9.77,6.604064,100.0
13143,2018-01-03,UNVR.JK,11175.0,11180.0,10640.0,9549.849609,13805500.0,2.0,240.790554,61.141268,83.044326,9.15,24.801895,0.0
13147,2018-01-04,UNVR.JK,10680.0,10800.0,10655.0,9443.741211,9651500.0,3.0,211.039859,58.257800,50.807092,9.22,10.333796,0.0
13151,2018-01-05,UNVR.JK,10680.0,10800.0,10635.0,9549.849609,8882500.0,4.0,193.790385,60.199544,52.000243,9.22,3.931329,0.0
13163,2018-01-08,UNVR.JK,10800.0,10940.0,10800.0,9620.590820,6800000.0,0.0,183.710576,61.436735,70.462063,9.52,8.029033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNVR.JK,8350.0,8420.0,8200.0,7616.660156,13365500.0,3.0,-49.981819,41.652679,-139.322678,13.94,3.749015,0.0
15979,2019-12-13,UNVR.JK,8280.0,8300.0,8200.0,7658.458984,12891500.0,4.0,-56.350273,42.737208,-151.408554,12.63,0.593755,0.0
15991,2019-12-16,UNVR.JK,8250.0,8275.0,8180.0,7598.082520,12503000.0,0.0,-65.513991,41.582279,-167.378990,12.14,0.376882,0.0
15995,2019-12-17,UNVR.JK,8180.0,8225.0,8180.0,7612.015625,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.29,0.019867,0.0


####String Action Version

In [179]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi4/5_DDPG/df_states_and_actions_str_ddpg_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,JECC.JK
0,13136,2018-01-02,4700.0,4700.0,4700.0,4347.121094,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604064,Tahan
1,13140,2018-01-03,4700.0,4700.0,4700.0,4347.121094,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801895,Tahan
2,13144,2018-01-04,4700.0,4700.0,4700.0,4347.121094,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333796,Tahan
3,13148,2018-01-05,4700.0,4700.0,4700.0,4347.121094,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931329,Tahan
4,13160,2018-01-08,5100.0,5175.0,5100.0,4786.458008,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029033,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,6175.0,6175.0,6175.0,6175.000000,0.0,3.0,-3.619868,53.058014,-40.969081,13.94,3.749015,Tahan
491,15976,2019-12-13,6175.0,6175.0,6175.0,6175.000000,0.0,4.0,-3.380631,53.058014,-139.322678,12.63,0.593755,Tahan
492,15988,2019-12-16,6175.0,6175.0,6175.0,6175.000000,0.0,0.0,-3.154669,53.058014,-151.408554,12.14,0.376882,Tahan
493,15992,2019-12-17,6175.0,6175.0,6175.0,6175.000000,0.0,1.0,-2.941682,53.058014,-167.378990,12.29,0.019867,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,TMAS.JK
0,13137,2018-01-02,229.0,229.0,218.0,176.273712,16500.0,1.0,-2.067101,43.270800,-110.403899,9.77,6.604064,Beli 54.0 saham
1,13141,2018-01-03,228.0,236.0,219.0,189.211227,27500.0,2.0,-1.382511,51.234612,-13.715756,9.15,24.801895,Tahan
2,13145,2018-01-04,239.0,239.0,217.0,182.742493,15500.0,3.0,-1.346421,47.766229,-35.329556,9.22,10.333796,Tahan
3,13149,2018-01-05,220.0,230.0,220.0,185.976868,25000.0,4.0,-1.044788,49.533335,-44.898608,9.22,3.931329,Tahan
4,13161,2018-01-08,229.0,233.0,228.0,188.402649,2500.0,0.0,-0.603051,50.824102,28.633429,9.52,8.029033,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,99.0,100.0,97.0,78.433716,3130800.0,3.0,-3.753758,35.769412,-110.875793,13.94,3.749015,Tahan
491,15977,2019-12-13,98.0,100.0,98.0,80.050911,1143700.0,4.0,-3.607023,37.844101,-93.126705,12.63,0.593755,Tahan
492,15989,2019-12-16,100.0,100.0,98.0,79.242325,1726300.0,0.0,-3.515456,37.222229,-89.104401,12.14,0.376882,Tahan
493,15993,2019-12-17,99.0,99.0,97.0,79.242325,2965500.0,1.0,-3.403654,37.222229,-89.887110,12.29,0.019867,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,TPIA.JK
0,13138,2018-01-02,1506.25,1506.25,1437.50,1444.110596,22387200.0,1.0,19.810029,60.719825,142.844563,9.77,6.604064,Tahan
1,13142,2018-01-03,1481.25,1487.50,1425.00,1407.550659,16574000.0,2.0,18.517521,56.440260,85.907381,9.15,24.801895,Tahan
2,13146,2018-01-04,1443.75,1443.75,1375.00,1352.711060,21859600.0,3.0,12.919177,50.876154,-26.782100,9.22,10.333796,Tahan
3,13150,2018-01-05,1387.50,1443.75,1375.00,1389.270996,27563200.0,4.0,11.302246,54.003430,1.727055,9.22,3.931329,Tahan
4,13162,2018-01-08,1431.25,1443.75,1406.25,1401.457397,25656000.0,0.0,10.878753,54.991442,33.184555,9.52,8.029033,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,2462.50,2487.50,2450.00,2440.179688,32832800.0,3.0,37.659417,62.950307,136.531575,13.94,3.749015,Tahan
491,15978,2019-12-13,2468.75,2487.50,2462.50,2452.566406,25854400.0,4.0,40.175702,63.571498,132.220628,12.63,0.593755,Tahan
492,15990,2019-12-16,2475.00,2493.75,2462.50,2458.759521,20370000.0,0.0,42.183346,63.884688,123.628220,12.14,0.376882,Tahan
493,15994,2019-12-17,2481.25,2506.25,2475.00,2464.952881,20856400.0,1.0,43.769622,64.203073,122.159603,12.29,0.019867,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,UNVR.JK
0,13139,2018-01-02,11235.0,11620.0,11000.0,9881.443359,12108500.0,1.0,264.457087,71.890166,158.326657,9.77,6.604064,Beli 100.0 saham
1,13143,2018-01-03,11175.0,11180.0,10640.0,9549.849609,13805500.0,2.0,240.790554,61.141268,83.044326,9.15,24.801895,Tahan
2,13147,2018-01-04,10680.0,10800.0,10655.0,9443.741211,9651500.0,3.0,211.039859,58.257800,50.807092,9.22,10.333796,Tahan
3,13151,2018-01-05,10680.0,10800.0,10635.0,9549.849609,8882500.0,4.0,193.790385,60.199544,52.000243,9.22,3.931329,Tahan
4,13163,2018-01-08,10800.0,10940.0,10800.0,9620.590820,6800000.0,0.0,183.710576,61.436735,70.462063,9.52,8.029033,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,8350.0,8420.0,8200.0,7616.660156,13365500.0,3.0,-49.981819,41.652679,-139.322678,13.94,3.749015,Tahan
491,15979,2019-12-13,8280.0,8300.0,8200.0,7658.458984,12891500.0,4.0,-56.350273,42.737208,-151.408554,12.63,0.593755,Tahan
492,15991,2019-12-16,8250.0,8275.0,8180.0,7598.082520,12503000.0,0.0,-65.513991,41.582279,-167.378990,12.14,0.376882,Tahan
493,15995,2019-12-17,8180.0,8225.0,8180.0,7612.015625,10837000.0,1.0,-70.835470,41.956738,-164.670408,12.29,0.019867,Tahan


<a id='7'></a>
# Bagian 8: Performa Backtesting
Backtesting memiliki peran kunci dalam mengevaluasi kinerja strategi perdagangan. Alat backtesting otomatis lebih disukai karena dapat meminimalisir kesalahan manusia. Backtesting dapat dilakukan dengan menggunakan paket Quantopian pyfolio untuk menguji strategi perdagangan pada notebook ini. Backtesting tersebut mudah digunakan dan terdiri dari berbagai plot yang memberikan gambaran komprehensif tentang kinerja strategi perdagangan.

<a id='7.1'></a>
## 8.1 Status BackTesting


###8.1.1 Agen A2C

In [180]:
print("===========Get Backtest Results of A2C Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_a2c)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_a2c_"+now+'.csv')

===========Get Backtest Results of A2C Model===========
Annual return          0.210779
Cumulative returns     0.456006
Annual volatility      0.399009
Sharpe ratio           0.675474
Calmar ratio           0.613203
Stability              0.353043
Max drawdown          -0.343735
Omega ratio            1.358304
Sortino ratio          1.147156
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.596878
Daily value at risk   -0.049201
dtype: float64


###8.1.2 Agen ppo

In [181]:
print("===========Get Backtest Results of PPO Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ppo)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ppo_"+now+'.csv')

===========Get Backtest Results of PPO Model===========
Annual return          0.071349
Cumulative returns     0.144967
Annual volatility      0.348985
Sharpe ratio           0.368611
Calmar ratio           0.205940
Stability              0.071414
Max drawdown          -0.346455
Omega ratio            1.199463
Sortino ratio          0.622041
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.710836
Daily value at risk   -0.043458
dtype: float64


###8.1.3 Agen TD3

In [182]:
print("===========Get Backtest Results of TD3 Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_td3)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_td3_"+now+'.csv')

===========Get Backtest Results of TD3 Model===========
Annual return          0.337089
Cumulative returns     0.769355
Annual volatility      0.359756
Sharpe ratio           0.986486
Calmar ratio           1.035472
Stability              0.279835
Max drawdown          -0.325541
Omega ratio            1.212540
Sortino ratio          1.560716
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.200520
Daily value at risk   -0.043917
dtype: float64


###8.1.4 Agen SAC

In [183]:
print("===========Get Backtest Results of SAC Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_sac)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_sac_"+now+'.csv')

===========Get Backtest Results of SAC Model===========
Annual return         -0.127695
Cumulative returns    -0.235362
Annual volatility      0.251555
Sharpe ratio          -0.418582
Calmar ratio          -0.442111
Stability              0.162439
Max drawdown          -0.288830
Omega ratio            0.928289
Sortino ratio         -0.589270
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.988638
Daily value at risk   -0.032111
dtype: float64


###8.1.5 Agen DDPG

In [184]:
print("===========Get Backtest Results of DDPG Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ddpg)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ddpg_"+now+'.csv')

===========Get Backtest Results of DDPG Model===========
Annual return         -0.127695
Cumulative returns    -0.235362
Annual volatility      0.251555
Sharpe ratio          -0.418582
Calmar ratio          -0.442111
Stability              0.162439
Max drawdown          -0.288830
Omega ratio            0.928289
Sortino ratio         -0.589270
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.988638
Daily value at risk   -0.032111
dtype: float64
